# **Notebook Listados OC SME Acquisition**
## Inicialización
1. Drive setup
2. Definición de funciones

---

## Listados de SUP1
SUP 1: Regular y DiDinversión


## Listados de SUP 2
SUP 2: Regular y Turbo
1. Seteo de inputs
2. Construcción
3. Ejecución

## Listados de ELC
ELC: Regular y Turbo (No se pregunta al Rs)




# Inicialización

## Drive Setup

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

from openpyxl import load_workbook

import pandas as pd
import os

from datetime import datetime, timedelta

creds, _ = default()
gc = gspread.authorize(creds)

from google.colab import auth
auth.authenticate_user()

Mounted at /content/drive


## Definición de funciones

In [ ]:
def upload(df, country, name, date, dry_run):
    if dry_run == True:
      output_directory = f"/content/drive/MyDrive/Notebook Listados/Run Tests/" # DRY RUN (GUARDA LISTADOS EN LA CARPETA DE PRUEBAS)
    else:
      output_directory = "/content/drive/Shared drives/Analytics_RCentral/Promos Automation/Acquisition/Events/XLSX/" # RUN REAL (GUARDA LISTADOS EN LA CARPETA DEL EVENTS)

    filename = f"{output_directory}{country}_{name}_{date}.csv"

    columna2 = pd.concat([pd.Series([df['Shop ID'].iloc[0]]), df['Shop ID']], ignore_index=True)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    columna2.to_csv(filename, index=False, header=False)
    print(f"✅ {country}_{name}_{date}, se invitaron ", len(df), " restaurantes al evento")

In [ ]:
def generarListadosSupElc(df, evento, countries, date, dry_run=False):
    if dry_run==True:
      print("Ejecutando DRY RUN | Los listados se guardaran en la carpeta de pruebas\n")
    else:
      print("Generando listados | Los listados creados se guardaran en la carpeta del Events Automation\n")
    mx = []
    co = []
    pe = []
    cr = []
    CC = [mx, co, pe, cr]

    for c in countries:
      filtradoTurbo = df[(df['Turbo?'] == True) & (df['Pais'] == c)]
      if not filtradoTurbo.empty:
          upload(filtradoTurbo, c, f'{evento}_TURBO', date, dry_run)
          if c == 'MX': mx.append(f"{c}_{evento}_TURBO_{date}.csv")
          if c == 'CO': co.append(f"{c}_{evento}_TURBO_{date}.csv")
          if c == 'PE': pe.append(f"{c}_{evento}_TURBO_{date}.csv")
          if c == 'CR': cr.append(f"{c}_{evento}_TURBO_{date}.csv")

      filtradoNoTurbo = df[(df['Turbo?'] == False) & (df['Pais'] == c)]
      if not filtradoNoTurbo.empty:
          upload(filtradoNoTurbo, c, evento, date, dry_run)
          if c == 'MX': mx.append(f"{c}_{evento}_{date}.csv")
          if c == 'CO': co.append(f"{c}_{evento}_{date}.csv")
          if c == 'PE': pe.append(f"{c}_{evento}_{date}.csv")
          if c == 'CR': cr.append(f"{c}_{evento}_{date}.csv")

    print("Listados generados correctamente!\n")
    for l in CC:
      for e in l:
        print(f"{e}")
    numEv = (len(mx)+len(co)+len(pe)+len(cr))
    print(f'\nHay {numEv} eventos este mes!')

In [ ]:
from openpyxl import load_workbook

def createSupElcTemplate(content, name, directory, SUP1 = False):
  if SUP1:
    template = load_workbook('/content/drive/MyDrive/Notebook Listados/sup1template.xlsx')
  else:
    template = load_workbook('/content/drive/MyDrive/Notebook Listados/suptemplate.xlsx')
  templateSheet = template['Worksheet']

  data = content.values.tolist()

  startRow = 2
  startCol = 1

  for i, fila in enumerate(data, start=startRow):
    for j, valor in enumerate(fila, start=startCol):
      templateSheet.cell(row=i, column=j, value=valor)

  full_path = f'{directory}{name}.xlsx'
  template.save(full_path)
  #files.download(full_path)  # <---- para descargarlo local


In [ ]:
def getSupElcTemplates(df, discounts, discountsTurbo, promoName, dryRun=True):
  if dryRun:
    output_directory = f"/content/drive/MyDrive/Notebook Listados/Run Tests/" # DRY RUN (GUARDA LISTADOS EN LA CARPETA DE PRUEBAS)
    print(f'Los templates se guardaran en la carpeta de pruebas')
  else:
    output_directory = "/content/drive/Shared drives/Analytics_RCentral/Promos Automation/Acquisition/Events/Templates/" # RUN REAL (GUARDA LISTADOS EN LA CARPETA DEL EVENTS)
    print(f'Los templates se guardaran en el Events Automation')

  invLists = {'CO': [], 'PE': [], 'CR': []}

  for c, d in discounts.items():
    regular = df[(df['Pais'] == c) & (df['Turbo?'] == False)]
    regular = regular[['shop_id', 'item_id']]
    regular['discount'] = d
    shops = regular['shop_id'].unique()
    if len(shops) > 0:
      templateName = getTemplateName(c, promoName)
      print(f'✅ {templateName} se registraran {len(regular)} items en {len(shops)} Rs para SUP 2')
      createSupElcTemplate(regular, templateName, output_directory)

  for c, d in discountsTurbo.items():
    turbo = df[(df['Pais'] == c) & (df['Turbo?'] == True)]
    turbo = turbo[['shop_id', 'item_id']]
    turbo['discount'] = d
    shopsT = turbo['shop_id'].unique()
    if len(shopsT) > 0:
      templateName = getTemplateName(c, f'{promoName}_TURBO')
      print(f'✅ {templateName} se registraran {len(turbo)} items en {len(shopsT)} Rs para SUP 2 Turbo')
      createSupElcTemplate(turbo, templateName, output_directory)

In [ ]:
def getSup1Templates(df, discounts, discountsDI, promoName, dryRun=True):
  if dryRun:
    output_directory = f"/content/drive/MyDrive/Notebook Listados/Run Tests/" # DRY RUN (GUARDA LISTADOS EN LA CARPETA DE PRUEBAS)
    print(f'Los templates se guardaran en la carpeta de pruebas')
  else:
    output_directory = "/content/drive/Shared drives/Analytics_RCentral/Promos Automation/Acquisition/Events/Templates/" # RUN REAL (GUARDA LISTADOS EN LA CARPETA DEL EVENTS)
    print(f'Los templates se guardaran en el Events Automation')

  invLists = {'CO': [], 'PE': [], 'CR': []}

  for c in ['CO','PE','CR']:
    for cc, d in discounts.items():
      if cc == c:
        regular = df[(df['Pais'] == c) & (df['DiDinversion?']==False)].copy()
        regular['discount'] = d[0]
        regular['subsidy'] = d[1]
        #print(regular) #----------------------------------
        regular = regular[['shop_id', 'item_id', 'discount', 'subsidy']]

    for cc, d in discountsDI.items():
      if cc == c:
        dinv = df[(df['Pais'] == c) & (df['DiDinversion?']==True)].copy()
        dinv['discount'] = d[0]
        dinv['subsidy'] = d[1]
        #print(dinv) #-----------------------------------
        dinv = dinv[['shop_id', 'item_id', 'discount', 'subsidy']]

    allDishes = pd.concat([regular, dinv], ignore_index=True)

    shops = len(regular['shop_id'].unique())+len(dinv['shop_id'].unique())
    if shops > 0:
        templateName = getTemplateName(c, promoName)
        print(f'✅ {templateName} se registraran {len(allDishes)} items en {shops} Rs para SUP 1 ({len(dinv['shop_id'].unique())} Rs con DiDinversión)')
        createSupElcTemplate(allDishes, templateName, output_directory, SUP1= True)

In [ ]:
def getTemplateName(cc, type):
    hoy = datetime.today()
    dia = f'{hoy.day:02d}'
    mes = f'{hoy.month:02d}'
    anio = f'{hoy.year:02d}'
    name = f'{mes}{dia}{anio}_Reg{type}_{cc}'
    return name

# SUP 1


`Cargar fecha de SUP 1 correspondiente` &rarr; `Carga de New Promo Tracker` &rarr; `Limpieza y filtrado de New Promo Tracker (Corresponding Rs)` &rarr; `Carga de Forms SSL SUP` &rarr; `Limpieza y filtrado de Forms SSL SUP` &rarr; `Join entre Forms y CRs (Invites)` &rarr; `Discriminación  entre oferta DiDinversión-Regular` &rarr; **`Generación de listados SUP1`** &rarr; `Transposición de dishes` &rarr; `Mapeo de descuentos y subsidios` &rarr; **`Generación de templates SUP1`**


## Seteo de inputs

In [ ]:
# Cargar fecha de SUP 1 correspondiente

date = '2025-12-23'
# formato 'AAAA-MM-DD'

In [ ]:
# Mapeo de descuentos y subsidios
import numpy as np
discountsRewards = {
    "CO" : [35, 100],
    "PE" : [30, 90],
    "CR" : [30, 100]
}

discountsRewardsDI = {
    "CO" : [35, 80],
    "PE" : [np.nan,np.nan],
    "CR" : [35, 80]
}


## Construcción

In [ ]:
# Carga de New Promo Tracker
creds, _ = default()
gc = gspread.authorize(creds)

sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/11aWZIXrsLxZrIFQguT6E7wCz_VxvxkG0HCIr_z-l-YQ/")
ws = sh.worksheet("New PromoTracker")


raw = ws.get_all_values()
headers = raw[5]
rows = raw[6:]

pT = pd.DataFrame(rows, columns=headers)

pT


,Shop ID,Nombre,Pais,Ciudad,Hunter,BA,District Name,BDM,Equipo,Store Type,...,SUP3 Signup,ELC Invite Date,ELC End Date,ELC Event,ELC Signup,Farmer Name,Narvarte,NewRBoost,Uber reviews,Column 30
0,5764608606255972832,Deli Fresita,MX,Veracruz,dcastillolagunes,,,fmacouzetsarti,BD,Postulado,...,FALSE,2025-06-13,2025-08-31,5765209490418762411,TRUE,,No,NO,#N/A,
1,5764608406703571355,Chicken Wings,CO,"Bogotá, D.C.",sandracasas,,,jserranouribe,BD,Postulado,...,FALSE,2025-06-13,2025-08-31,5765209259996283505,TRUE,,No,NO,#N/A,
2,5764608493299171760,Los Caudillos Sede La Nubia,CO,Medellín,maria.lozano.ol_v,,,No Hunter,Inbound,Postulado,...,FALSE,2025-06-13,2025-08-31,5765209259996283505,TRUE,,No,NO,#N/A,
3,5764608553202221495,Olam Sabores de Mar,CO,"Bogotá, D.C.",juan.santafe.ol_v,,,Andrés Jaimes,Inbound,Postulado,...,FALSE,2025-06-13,2025-08-31,5765209259996283505,TRUE,,No,NO,#N/A,
4,5764608560563225020,Rancho Burger,CO,Cali,sharith.r.ol_v,,,No Hunter,Inbound,Postulado,...,FALSE,2025-06-13,2025-08-31,5765209259996283505,TRUE,,No,NO,#N/A,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8308,5764610480883370577,K - Chebas Burguer,PE,Lima,kevin.olaya.ol_v,BA_9,BDArea 3 - D0 - Surquillo,No Hunter,Inbound,Postulado,...,FALSE,2026-01-16,Evento por Crear,Evento por Crear,FALSE,,No,NO,#N/A,#REF!
8309,5764610143636162669,Pizzas boneless & algo más,MX,Durango,magdiellunacorrales,BA_1,Area1,folivaresibarra,BD,Oferta B,...,FALSE,2026-02-10,Evento por Crear,Evento por Crear,FALSE,,No,NO,#N/A,#REF!
8310,5764610681685675941,Hot Wonder Wings,MX,Durango,magdiellunacorrales,BA_1,Area1,folivaresibarra,BD,Oferta B,...,FALSE,2026-02-10,Evento por Crear,Evento por Crear,FALSE,,No,NO,#N/A,#REF!
8311,5764610682050580049,Legado Cacao,CO,"Bogotá, D.C.",davidfariasalarcon,BA_1,BDArea 64,jserranouribe,BD,Postulado,...,FALSE,2026-01-13,Evento por Crear,Evento por Crear,FALSE,,No,NO,#N/A,#REF!


In [ ]:
# Limpieza y filtrado de New Promo Tracker

pT = pT[(pT['SUP 1 Date']==date) & (pT['Pais']!='MX') & (pT['Equipo']!='SOB')]

pT = pT[['Shop ID', 'Nombre', 'Pais', 'Ciudad', 'Hunter', 'BA', 'District Name',
       'BDM', 'Equipo', 'Store Type', 'SType Forms', 'Fecha de FO', 'SUP 1 Date']]

pT = pT.drop_duplicates()

len(pT)

#pT

16

In [ ]:
# Carga de Forms SSL SUP

ws2 = sh.worksheet("SSL SUP")


raw2 = ws2.get_all_values()
headers2 = raw2[0]
rows2 = raw2[1:]

forms = pd.DataFrame(rows2, columns=headers2)

forms.columns


Index(['Timestamp', 'Nombre de BD', 'País', 'Fecha de FO', 'Equipo', 'BDM',
       'Promoción a registrar',
       'Shop ID\n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697',
       '1. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN',
       '2. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN',
       '3. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN',
       '4. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\n

In [ ]:
# Limpieza y filtrado de Forms SSL
import numpy as np
forms = forms.rename(columns={'Shop ID\n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697':'Shop ID',
                              '1. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 1',
                              '2. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 2',
                              '3. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 3',
                              '4. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 4',
                              '5. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 5',
                              '6. Dish ID \nUNICAMENTE PARA COLOMBIA DIDINVERSION':'Dish 6'
                              })

forms_filtrado = forms[forms["Promoción a registrar"].str.contains(r"SUP\s*1", na=True)]
forms_filtrado['Dish 6'] = forms_filtrado['Dish 6'].astype(str)
forms_filtrado['Dish 6'] = forms_filtrado['Dish 6'].where(forms_filtrado["Dish 6"].str.fullmatch(r"\d{19}"), np.nan)
forms_filtrado["Timestamp"] = pd.to_datetime(forms_filtrado["Timestamp"], format="%m/%d/%Y %H:%M:%S")
forms_filtrado["Shop ID"] = forms_filtrado["Shop ID"].str.strip()

forms_filtrado



/tmp/ipython-input-3087232282.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forms_filtrado['Dish 6'] = forms_filtrado['Dish 6'].astype(str)
/tmp/ipython-input-3087232282.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forms_filtrado['Dish 6'] = forms_filtrado['Dish 6'].where(forms_filtrado["Dish 6"].str.fullmatch(r"\d{19}"), np.nan)
/tmp/ipython-input-3087232282.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Timestamp,Nombre de BD,País,Fecha de FO,Equipo,BDM,Promoción a registrar,Shop ID,Dish 1,Dish 2,Dish 3,Dish 4,Dish 5,Ciudad,Acepto las promos turbo?,Dish 6,BDM,Pais
6,2025-01-02 21:32:50,JOSE RAUL ESCOTO CORREA,MX,1/2/2025,BD,sandramartinez,SUP 1,5764607847573487950,5764700527887846787,5764700662831189375,5764700663380643199,5764700553691205004,5764700546133069200,Mazatlán,,NaN,,
7,2025-01-02 21:35:45,JOSE RAUL ESCOTO CORREA,MX,1/2/2025,BD,sandramartinez,SUP 1,5764608331608754124,5764700651808558023,5764700670775200719,5764700662726331343,5764700697325145034,5764700698017205195,Mazatlán,,NaN,,
8,2025-01-03 08:44:14,diego henao,CO,2/1/2025,BD,jserranouribe,SUP 1 Y SUP2 BD,5764608261744232219,5764700623526365649,5764700438251375072,5764700446191192529,5764700438498839008,5764700680317241832,Medellín,,NaN,,
9,2025-01-03 08:46:33,diego henao,CO,2/1/2025,BD,jserranouribe,SUP 1 Y SUP2 BD,5764608189346351167,5764700520627505708,5764700545956907531,5764700519457294856,5764700479674321424,5764700703843093008,Medellín,,NaN,,
10,2025-01-03 08:50:34,d.rojas.ol_v,CO,12/27/2024,INBOUND,,SUP 1 y SUP 2 INBOUND,5764608340139968302,5764700437643200345,5764700528345024336,5764700556178425664,5764700521000798021,5764700527279671124,Barranquilla,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14900,2025-12-23 11:14:45,Edrei Sosa Vazquez,MX ALL Dishes (Oferta A UBER Y Oferta B),12/22/2025,BD,hugodiazhurtado,SUP 1,5764609654152497318,,,,,,Monterrey,,NaN,,MX ALL Dishes (Oferta A UBER Y Oferta B)
14901,2025-12-23 11:15:04,Paola Macias,MX 5 DISHES,12/23/2025,BD,claudiaporrascastro,SUP 1,5764610442165750177,5764709247925553266,5764709284734765161,5764709344105138320,5764709412057057419,5764709499541850220,CDMX,Si,NaN,,MX 5 DISHES
14902,2025-12-23 11:16:15,Edrei Sosa Vazquez,MX ALL Dishes (Oferta A UBER Y Oferta B),12/22/2025,BD,hugodiazhurtado,SUP 1,5764610684500053945,,,,,,Monterrey,,NaN,,MX ALL Dishes (Oferta A UBER Y Oferta B)
14903,2025-12-23 11:16:43,Edrei Sosa Vazquez,MX ALL Dishes (Oferta A UBER Y Oferta B),12/22/2025,BD,hugodiazhurtado,SUP 1,5764610807699345332,,,,,,Monterrey,,NaN,,MX ALL Dishes (Oferta A UBER Y Oferta B)


In [ ]:
forms_last = (
    forms_filtrado
    .sort_values("Timestamp")
    .drop_duplicates(subset=["Shop ID"], keep="last")
)

len(forms_last)

12705

In [ ]:
# Join entre Forms y CRs Qualifier

SUP1file = pT.merge(
    forms_last[['Shop ID', 'País', 'Dish 1', 'Dish 2', 'Dish 3', 'Dish 4', 'Dish 5', 'Dish 6']],
    on="Shop ID",
    how="left"
)

SUP1file

,Shop ID,Nombre,Pais,Ciudad,Hunter,BA,District Name,BDM,Equipo,Store Type,SType Forms,Fecha de FO,SUP 1 Date,País,Dish 1,Dish 2,Dish 3,Dish 4,Dish 5,Dish 6
0,5764610600991461123,Nomada Comida Rapida,CO,Barranquilla,brayanhamburger,BA_2,BDArea 32,jserranouribe,BD,Postulado,CO Normal,2025-12-19,2025-12-23,CO Normal,5764709145605508839,5764709045906902785,5764709057751617271,5764709008552431366,5764709186084736768,NaN
1,5764610647342714400,Angelato´s Fast Food,CO,Cali,danielaramosvaron,BA_3,BDArea 23,jserranouribe,BD,Postulado,CO Normal,2025-12-20,2025-12-23,CO Normal,5764709236500269632,5764709285632346701,5764709387264526901,5764709322672245325,5764709286903220794,NaN
2,5764610683782827493,Hojaldraría 28 de Febrero,CO,Medellín,sararuedarueda,BA_1,BDArea 50,jserranouribe,BD,Postulado,CO Normal,2025-12-19,2025-12-23,CO Normal,5764709006551747836,5764708972582079769,5764709083525614859,5764709007587740957,5764709131219045629,NaN
3,5764610775403204598,Pollo y Parrilla mi Cali,CO,Cali,danielaramosvaron,BA_6,BDArea 31,jserranouribe,BD,Postulado,CO Normal,2025-12-20,2025-12-23,CO Normal,5764709243773192836,5764709338493160082,5764709489597155981,5764709449503804014,5764709451663870574,NaN
4,5764608119288892799,Field Fruits,CO,"Bogotá, D.C.",davidfariasalarcon,BA_3,BDArea 75,jserranouribe,BD,Postulado,No Forms,2025-12-20,2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5764610585527060530,Angelato´s Pizzeria,CO,Cali,danielaramosvaron,BA_3,BDArea 23,jserranouribe,BD,Postulado,CO Normal,2025-12-20,2025-12-23,CO Normal,5764709293077236304,5764709341919906384,5764709369883331158,5764709323381082691,5764709396487801423,NaN
6,5764610592162448363,Master Puppy,CO,"Bogotá, D.C.",mariapalaciosramos,BA_5,BDArea 50,No Hunter,BD,Postulado,No Forms,2025-12-19,2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5764610878578884654,Mara Clack Fast Food,CO,Cali,danielaramosvaron,BA_5,BDArea 8,jserranouribe,BD,Postulado,CO Normal,2025-12-20,2025-12-23,CO Normal,5764709239960570517,5764709323578215065,5764709438485367449,5764709436191083152,5764709379869969056,NaN
8,5764610723007958928,Don D Varo,CR,San Jose,wagnerblanco,BA-11,BDArea 67,josemauriciomontero,BD,Postulado,CR Normal,2025-12-19,2025-12-23,CR Normal,5764709424522527600,5764709323280417657,5764709309745398657,5764709467514143613,5764709360915909238,NaN
9,5764610733112036936,Raices del Pacifico,CO,Cali,danielaramosvaron,BA_4,BDArea 1,jserranouribe,BD,Postulado,No Forms,2025-12-19,2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Discriminación entre oferta DiDinversión-Regular

SUP1file['Turbo?'] = False
SUP1file['DiDinversion?'] = SUP1file['País'].str.contains("DiDinversión", na=False)

SUP1file[SUP1file['Pais']=='CO']


,Shop ID,Nombre,Pais,Ciudad,Hunter,BA,District Name,BDM,Equipo,Store Type,...,SUP 1 Date,País,Dish 1,Dish 2,Dish 3,Dish 4,Dish 5,Dish 6,Turbo?,DiDinversion?
0,5764610600991461123,Nomada Comida Rapida,CO,Barranquilla,brayanhamburger,BA_2,BDArea 32,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709145605508839,5764709045906902785,5764709057751617271,5764709008552431366,5764709186084736768,NaN,False,False
1,5764610647342714400,Angelato´s Fast Food,CO,Cali,danielaramosvaron,BA_3,BDArea 23,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709236500269632,5764709285632346701,5764709387264526901,5764709322672245325,5764709286903220794,NaN,False,False
2,5764610683782827493,Hojaldraría 28 de Febrero,CO,Medellín,sararuedarueda,BA_1,BDArea 50,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709006551747836,5764708972582079769,5764709083525614859,5764709007587740957,5764709131219045629,NaN,False,False
3,5764610775403204598,Pollo y Parrilla mi Cali,CO,Cali,danielaramosvaron,BA_6,BDArea 31,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709243773192836,5764709338493160082,5764709489597155981,5764709449503804014,5764709451663870574,NaN,False,False
4,5764608119288892799,Field Fruits,CO,"Bogotá, D.C.",davidfariasalarcon,BA_3,BDArea 75,jserranouribe,BD,Postulado,...,2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
5,5764610585527060530,Angelato´s Pizzeria,CO,Cali,danielaramosvaron,BA_3,BDArea 23,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709293077236304,5764709341919906384,5764709369883331158,5764709323381082691,5764709396487801423,NaN,False,False
6,5764610592162448363,Master Puppy,CO,"Bogotá, D.C.",mariapalaciosramos,BA_5,BDArea 50,No Hunter,BD,Postulado,...,2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
7,5764610878578884654,Mara Clack Fast Food,CO,Cali,danielaramosvaron,BA_5,BDArea 8,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709239960570517,5764709323578215065,5764709438485367449,5764709436191083152,5764709379869969056,NaN,False,False
9,5764610733112036936,Raices del Pacifico,CO,Cali,danielaramosvaron,BA_4,BDArea 1,jserranouribe,BD,Postulado,...,2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
11,5764610582343585701,Sason 87,CO,Barranquilla,brayanhamburger,BA_2,BDArea 34,jserranouribe,BD,Postulado,...,2025-12-23,CO Normal,5764709258717497324,5764709350606308247,5764709311209210731,5764709234507973482,5764709234805769066,NaN,False,False


In [ ]:

SUP1file.loc[SUP1file['Pais'] == 'PE', 'DiDinversion?'] = False
SUP1file.loc[SUP1file['DiDinversion?'] == False, 'Dish 6'] = np.nan


In [ ]:
# Generación de listados

In [ ]:
# Transposición de dishes

df_items_list = []

dish_cols = [c for c in SUP1file.columns if c.lower().startswith("dish")]

dish_cols


['Dish 1', 'Dish 2', 'Dish 3', 'Dish 4', 'Dish 5', 'Dish 6']

In [ ]:

for _, row in SUP1file.iterrows():
    pais = row["Pais"]
    shop = row["Shop ID"]
    turbo = row["DiDinversion?"]

    for col in dish_cols:
        item = row[col]

        # validar que sí existe un platillo
        if pd.notna(item) and str(item).strip() != "":
            df_items_list.append({
                "Pais": pais,
                "shop_id": shop,
                "DiDinversion?": turbo,
                "item_id": item
            })

df_items = pd.DataFrame(df_items_list)

df_items[df_items['Pais']=='PE']
#df_items.to_csv("archivobase.csv", index=False)
#files.download("archivobase.csv")


,Pais,shop_id,DiDinversion?,item_id


In [ ]:
# Mapeo de descuentos y subsidios

## Ejecución

In [ ]:
# Generación de listados SUP 1

generarListadosSupElc(SUP1file, 'SUP1', ['CO','PE','CR'], date, dry_run=False)

Generando listados | Los listados creados se guardaran en la carpeta del Events Automation

✅ CO_SUP1_2025-12-23, se invitaron  13  restaurantes al evento
✅ PE_SUP1_2025-12-23, se invitaron  1  restaurantes al evento
✅ CR_SUP1_2025-12-23, se invitaron  2  restaurantes al evento
Listados generados correctamente!

CO_SUP1_2025-12-23.csv
PE_SUP1_2025-12-23.csv
CR_SUP1_2025-12-23.csv

Hay 3 eventos este mes!


In [ ]:
# Generación de templates SUP 1

getSup1Templates(df_items, discountsRewards, discountsRewardsDI, 'SUP1', dryRun =False)

Los templates se guardaran en el Events Automation
✅ 12232025_RegSUP1_CO se registraran 45 items en 9 Rs para SUP 1 (0 Rs con DiDinversión)
✅ 12232025_RegSUP1_CR se registraran 10 items en 2 Rs para SUP 1 (1 Rs con DiDinversión)


# SUP 2

`Cargar fecha de SUP 2 correspondiente` &rarr; `Carga de dash de PH` &rarr; `Creación de pivot de dashPH` &rarr; `Carga de New Promo Tracker` &rarr; `Limpieza y filtrado de New Promo Tracker (Corresponding Rs)` &rarr; `Join entre PivotPH y Corresponding Rs (CRs Qualifier)` &rarr; `Carga de Forms SSL SUP` &rarr; `Limpieza y filtrado de Forms SSL SUP` &rarr; `Join entre Forms y CRs Qualifier (Invites)` &rarr; `Discriminación  entre oferta Turbo-Regular` &rarr; **`Generación de listados SUP2`** &rarr; `Transposición de dishes` &rarr; `Mapeo de descuentos` &rarr; **`Generación de templates SUP2`**


## Seteo de inputs

In [ ]:
# Cargar fecha de SUP 2 correspondiente

date = '2025-12-09'
# formato 'AAAA-MM-DD'

In [ ]:
# Carga dash de PH
listado = '/content/drive/MyDrive/Notebook Listados/Sup2Peak.xlsx'

dashPH = pd.read_excel(listado, dtype={'shop_id':str}, usecols = ['shop_id',	'Shop name',	'Peak 1',	'Peak 2',	'dt(周)',	'Peak coverage',	'online_rate', 'daily_set_hours'])


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# Mapeo de descuentos

discounts = {
    "CO" : 35,
    "PE" : 30,
    "CR" : 30,
}

discountsTurbo = {
    "CO" : 40,
    "PE" : 30,
    "CR" : 35
}

## Construcción

In [ ]:
# Creación de pivot de dashPH

pivot = dashPH.pivot_table(
    index="shop_id",
    values="Peak coverage",
    aggfunc="mean"
).reset_index()

#pivot

In [ ]:
# Carga de New Promo Tracker
creds, _ = default()
gc = gspread.authorize(creds)

sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/11aWZIXrsLxZrIFQguT6E7wCz_VxvxkG0HCIr_z-l-YQ/")
ws = sh.worksheet("New PromoTracker")


raw = ws.get_all_values()
headers = raw[5]
rows = raw[6:]

pT = pd.DataFrame(rows, columns=headers)

pT


In [ ]:
# Limpieza y filtrado de New Promo Tracker

pT = pT[(pT['SUP 2 Date']==date) & (pT['Pais']!='MX') & (pT['Equipo']!='SOB')]

pT = pT[['Shop ID', 'Nombre', 'Pais', 'Ciudad', 'Hunter', 'BA', 'District Name',
       'BDM', 'Equipo', 'Store Type', 'SType Forms', 'Fecha de FO', 'SUP 2 Date']]

pT = pT.drop_duplicates()

len(pT)

#pT

In [ ]:
# Join entre PivotPH y New Promo Tracker

pivot_clean = pivot[["shop_id", "Peak coverage"]].rename(columns={"shop_id": "Shop ID"})


crsq = pT.merge(
    pivot_clean,
    on="Shop ID",
    how="left"
)

# NaN -> 0
crsq["Peak coverage"] = crsq["Peak coverage"].fillna(0)

crsq

In [ ]:
# Carga de Forms SSL SUP

ws2 = sh.worksheet("SSL SUP")


raw2 = ws2.get_all_values()
headers2 = raw2[0]
rows2 = raw2[1:]

forms = pd.DataFrame(rows2, columns=headers2)

forms.columns


In [ ]:
# Limpieza y filtrado de Forms SSL

forms = forms.rename(columns={'Shop ID\n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697':'Shop ID',
                              '1. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 1',
                              '2. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 2',
                              '3. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 3',
                              '4. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 4',
                              '5. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 5',
                              #'6. Dish ID \nUNICAMENTE PARA COLOMBIA DIDINVERSION':'Dish 6'
                              })

forms_filtrado = forms[forms['País'].isin(['CO', 'CO DiDinversión', 'CO Normal', 'CR', 'CR DiDinversión', 'CR Normal', 'PE'])]
forms_filtrado = forms_filtrado[forms["Promoción a registrar"].str.contains(r"SUP\s*2", na=True)]
forms_filtrado["Timestamp"] = pd.to_datetime(forms_filtrado["Timestamp"], format="%m/%d/%Y %H:%M:%S")
forms_filtrado["Shop ID"] = forms_filtrado["Shop ID"].str.strip()



In [ ]:
forms_last = (
    forms_filtrado
    .sort_values("Timestamp")
    .drop_duplicates(subset=["Shop ID"], keep="last")
)

len(forms_last)

In [ ]:
# Join entre Forms y CRs Qualifier

SUP2file = crsq.merge(
    forms_last[['Shop ID', 'Acepto las promos turbo?', 'Dish 1', 'Dish 2', 'Dish 3', 'Dish 4', 'Dish 5']],
    on="Shop ID",
    how="left"
)

SUP2file


In [ ]:
# Discriminación entre ofertas Turbo-Regular

SUP2file['Turbo?'] = (SUP2file['Peak coverage']>=0.95) & (SUP2file['Acepto las promos turbo?'] == 'Si')
SUP2file

In [ ]:
print(len(SUP2file[SUP2file['SType Forms'] == 'No Forms']))

print(len(SUP2file[SUP2file['Pais']=='CO']))
print(len(SUP2file[SUP2file['Pais']=='PE']))
print(len(SUP2file[SUP2file['Pais']=='CR']))
SUP2file[SUP2file['SType Forms'] == 'No Forms']

In [ ]:
# Generación de listados

In [ ]:
# Transposición de dishes

df_items_list = []



dish_cols = [c for c in SUP2file.columns if c.lower().startswith("dish")]
#dish_cols


In [ ]:

for _, row in SUP2file.iterrows():
    pais = row["Pais"]
    shop = row["Shop ID"]
    turbo = row["Turbo?"]

    for col in dish_cols:
        item = row[col]

        # validar que sí existe un platillo
        if pd.notna(item) and str(item).strip() != "":
            df_items_list.append({
                "Pais": pais,
                "shop_id": shop,
                "Turbo?": turbo,
                "item_id": item
            })

df_items = pd.DataFrame(df_items_list)

df_items

#df_items.to_csv("archivobase.csv", index=False)
#files.download("archivobase.csv")

## Ejecución

In [ ]:
# Generación de listados SUP 2

generarListadosSupElc(SUP2file, 'SUP2', ['CO','PE','CR'], date, dry_run=False)

Generando listados | Los listados creados se guardaran en la carpeta del Events Automation

✅ CO_SUP2_2025-12-09, se invitaron  6  restaurantes al evento
✅ PE_SUP2_2025-12-09, se invitaron  3  restaurantes al evento
✅ CR_SUP2_TURBO_2025-12-09, se invitaron  1  restaurantes al evento
✅ CR_SUP2_2025-12-09, se invitaron  8  restaurantes al evento
Listados generados correctamente!

CO_SUP2_2025-12-09.csv
PE_SUP2_2025-12-09.csv
CR_SUP2_TURBO_2025-12-09.csv
CR_SUP2_2025-12-09.csv

Hay 4 eventos este mes!


In [ ]:
# Generación de templates SUP 2

getSupElcTemplates(df_items, discounts, discountsTurbo, 'SUP2', dryRun = True)

In [ ]:
# Descargar el archivo base
SUP2file.to_csv("archivobase.csv", index=False)
files.download("archivobase.csv")

# recargas aprovechando este archivo !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# Descargar el archivo de dishes
df_items.to_csv("archivodishes.csv", index=False)
files.download("archivodishes.csv")

# ELC


`Cargar fecha de ELC correspondiente` &rarr; `Carga de output de AVG DO Q` &rarr; `Carga de New Promo Tracker` &rarr; `Limpieza y filtrado de New Promo Tracker (Corresponding Rs)` &rarr; `Construcción de qualifier en AVG DO` &rarr; `Join entre AVG DO y Corresponding Rs` &rarr; *`Discriminación  entre oferta Turbo-Regular`* &rarr; **`Generación de listados ELC`**

## Seteo de inputs

In [ ]:
# Cargar fecha de ELC correspondiente

date = '2025-12-09'
# formato 'AAAA-MM-DD'

In [ ]:
# Cargar query AVG DO

dailyorders = '/content/drive/MyDrive/Notebook Listados/QUALIFIERELCTURBO9DIC.xlsx'

avgdo = pd.read_excel(dailyorders, dtype={'shop_id':str}, usecols = ['shop_id', 'fo_date', 'country_code',	'orders_29', 'online_days_29'])

#avgdo

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Construcción

In [ ]:
# Carga de New Promo Tracker

creds, _ = default()
gc = gspread.authorize(creds)

sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/11aWZIXrsLxZrIFQguT6E7wCz_VxvxkG0HCIr_z-l-YQ/")
ws = sh.worksheet("New PromoTracker")


raw = ws.get_all_values()
headers = raw[5]
rows = raw[6:]

pT = pd.DataFrame(rows, columns=headers)

#pT


In [ ]:
# Limpieza y filtrado de New Promo Tracker

pT = pT[(pT['ELC Invite Date']==date) & (pT['Pais']!='MX') & (pT['Equipo']!='SOB')]

pT = pT[['Shop ID', 'Nombre', 'Pais', 'Ciudad', 'Hunter', 'BA', 'District Name',
       'BDM', 'Equipo', 'Store Type', 'Fecha de FO', 'ELC Invite Date']]

pT = pT.drop_duplicates()

len(pT)

#pT

28

In [ ]:
# Construccion de qualifier en AVG DO Q

avgdo['Qualifier'] = avgdo['orders_29'] / avgdo['online_days_29']

avgdo['Turbo?'] = avgdo['Qualifier'] >= 5

avgdo

,shop_id,fo_date,country_code,orders_29,online_days_29,Qualifier,Turbo?
0,5764610380706611526,2025-11-18,MX,22,8,2.750000,False
1,5764610278705334600,2025-11-21,MX,17,12,1.416667,False
2,5764610296761811023,2025-11-12,MX,11,15,0.733333,False
3,5764610169405966942,2025-11-14,MX,8,23,0.347826,False
4,5764610424503534109,2025-11-27,MX,9,8,1.125000,False
...,...,...,...,...,...,...,...
5034,5764610356245432940,2025-11-24,MX,0,1,0.000000,False
5035,5764610398607903500,2025-12-01,MX,0,5,0.000000,False
5036,5764609189008375958,2025-12-06,MX,1,2,0.500000,False
5037,5764610473425898662,2025-12-05,MX,0,0,NaN,False


In [ ]:
# Join entre AVG DO y Corresponding Rs

avgdo = avgdo.rename(columns={"shop_id": "Shop ID"})

ELCfile = pT.merge(
    avgdo,
    on="Shop ID",
    how="left"
)

ELCfile['Turbo?'] = ELCfile['Turbo?'].fillna(False)

ELCfile

/tmp/ipython-input-4285212543.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ELCfile['Turbo?'] = ELCfile['Turbo?'].fillna(False)


,Shop ID,Nombre,Pais,Ciudad,Hunter,BA,District Name,BDM,Equipo,Store Type,Fecha de FO,ELC Invite Date,fo_date,country_code,orders_29,online_days_29,Qualifier,Turbo?
0,5764610053446045647,El Faisán Chino,CO,Cali,monicamena,BA_4,BDArea 4,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,33.0,21.0,1.571429,False
1,5764610228914751840,Dórilocos Donde Fuya,CO,"Bogotá, D.C.",sergioninobermudez,BA_7,BDArea 0,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,3.0,7.0,0.428571,False
2,5764610524441215213,Delicias Arias,CO,Cali,jinethrojas,BA_4,BDArea 14,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,4.0,6.0,0.666667,False
3,5764607790992329960,Nando Rojas Butifarras,CO,Barranquilla,brayanhamburger,BA_2,BDArea 5,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,29.0,19.0,1.526316,False
4,5764610387987923231,La Receta Dorada,CO,Cali,danielaramosvaron,BA_3,BDArea 17,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,51.0,22.0,2.318182,False
5,5764610503528415471,Al Biche,CO,Cali,jinethrojas,BA_6,BDArea 30,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,17.0,22.0,0.772727,False
6,5764610545337237725,Lechoneria La Mas,CO,Cali,jinethrojas,BA_4,BDArea 14,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,183.0,17.0,10.764706,True
7,5764610132835831287,Restaurante KR Artesano del Sabor,CO,"Bogotá, D.C.",anyelosierrarico,BA_5,BDArea 42,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,63.0,18.0,3.500000,False
8,5764610289451139308,Pizzetas JL,CO,Barranquilla,brayanhamburger,BA_1,BDArea 39,jserranouribe,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,CO,24.0,18.0,1.333333,False
9,5764610262112667732,Salchilandia,PE,Lima,ivanbarcochrem,BA_3,BDArea 0 - D0 - Lince,omaramayo,BD,Postulado,2025-11-14,2025-12-09,2025-11-14,PE,3.0,8.0,0.375000,False


In [ ]:
# Generación de listados

## Ejecución

In [ ]:
# Generación de listados ELC

generarListadosSupElc(ELCfile, 'ELC', ['CO','PE','CR'], date, dry_run=False)

Generando listados | Los listados creados se guardaran en la carpeta del Events Automation

✅ CO_ELC_TURBO_2025-12-09, se invitaron  2  restaurantes al evento
✅ CO_ELC_2025-12-09, se invitaron  18  restaurantes al evento
✅ PE_ELC_2025-12-09, se invitaron  3  restaurantes al evento
✅ CR_ELC_2025-12-09, se invitaron  5  restaurantes al evento
Listados generados correctamente!

CO_ELC_TURBO_2025-12-09.csv
CO_ELC_2025-12-09.csv
PE_ELC_2025-12-09.csv
CR_ELC_2025-12-09.csv

Hay 4 eventos este mes!


# SOB

In [ ]:
from google.colab import drive
import os
import shutil
from datetime import datetime

# Montar Drive
drive.mount('/content/drive')

# Ruta base, aquí se guardan los listasdos
base_folder = '/content/drive/MyDrive/Events creation'
archivos = os.listdir(base_folder)

# Mostrar archivos, se muestran los files actuales en la carpeta (si está vació no imprime nada)
print("Contenido actual en la carpeta:")
for archivo in archivos:
    print(f" - {archivo}")

# Filtrar archivos CSV y obtener sus fechas de modificación para saber cuales son los más recientes
csv_archivos = [f for f in archivos if f.endswith('.csv')]
if not csv_archivos:
    print("\n⚠️ No hay archivos CSV en la carpeta.")
else:
    # Obtener el archivo CSV más reciente por fecha de modificación
    archivo_mas_reciente = max(csv_archivos, key=lambda f: os.path.getmtime(os.path.join(base_folder, f)))
    fecha_mod = datetime.fromtimestamp(os.path.getmtime(os.path.join(base_folder, archivo_mas_reciente)))
    fecha_formateada = fecha_mod.strftime('%Y-%m-%d')
#Para saber si los resultados mover a una carpeta historial o repositorio
    respuesta = input(f"\n¿Quieres mover los datos existentes a una carpeta con la fecha del último CSV ({fecha_formateada})? (1 = Sí, 0 = No): ")

    if respuesta == "1":
        # Crear carpeta con esa fecha en dado caso que si los quisiera guardar
        nueva_carpeta = os.path.join(base_folder, fecha_formateada)
        os.makedirs(nueva_carpeta, exist_ok=True)

        # Mover archivos (excepto la carpeta destino)
        for archivo in archivos:
            origen = os.path.join(base_folder, archivo)
            destino = os.path.join(nueva_carpeta, archivo)

            if origen == nueva_carpeta:
                continue

            try:
                shutil.move(origen, destino)
                print(f"✔️ Movido: {archivo}")
            except Exception as e:
                print(f"❌ Error al mover {archivo}: {e}")

        print("\n✅ Movimiento completo. Nuevo contenido en la carpeta:")
        print(os.listdir(nueva_carpeta))

        print("\n📁 La carpeta original ahora contiene:")
        print(os.listdir(base_folder))
    else:
        print("\nNo se movió nada.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Events creation'

## Funciones madre

In [ ]:
def getSUP2lists(pT, forms, date, dash, discounts, dry_run=False):
  # Creación de pivot de dashPH

  pivot = dashPH.pivot_table(
      index="shop_id",
      values="Peak coverage",
      aggfunc="mean"
  ).reset_index()

  # Limpieza y filtrado de New Promo Tracker

  pT = pT[(pT['SUP 2 Date']==date) & (pT['Pais']!='MX') & (pT['Equipo']!='SOB')]

  pT = pT[['Shop ID', 'Nombre', 'Pais', 'Ciudad', 'Hunter', 'BA', 'District Name',
         'BDM', 'Equipo', 'Store Type', 'SType Forms', 'Fecha de FO', 'SUP 2 Date']]

  pT = pT.drop_duplicates()

  # Join entre PivotPH y New Promo Tracker

  pivot_clean = pivot[["shop_id", "Peak coverage"]].rename(columns={"shop_id": "Shop ID"})


  crsq = pT.merge(
      pivot_clean,
      on="Shop ID",
      how="left"
  )

  # NaN -> 0
  crsq["Peak coverage"] = crsq["Peak coverage"].fillna(0)

  # Limpieza y filtrado de Forms SSL

  forms_filtrado = forms[forms['País'].isin(['CO', 'CO DiDinversión', 'CO Normal', 'CR', 'CR DiDinversión', 'CR Normal', 'PE'])]
  forms_filtrado = forms_filtrado[forms["Promoción a registrar"].str.contains(r"SUP\s*2", na=True)]
  forms_filtrado["Timestamp"] = pd.to_datetime(forms_filtrado["Timestamp"], format="%m/%d/%Y %H:%M:%S")
  forms_filtrado["Shop ID"] = forms_filtrado["Shop ID"].str.strip()

  forms_last = (
      forms_filtrado
      .sort_values("Timestamp")
      .drop_duplicates(subset=["Shop ID"], keep="last")
  )

  # Join entre Forms y CRs Qualifier

  SUP2file = crsq.merge (
      forms_last[['Shop ID', 'Acepto las promos turbo?', 'Dish 1', 'Dish 2', 'Dish 3', 'Dish 4', 'Dish 5']],
      on="Shop ID",
      how="left"
  )

  # Discriminación entre ofertas Turbo-Regular

  SUP2file['Turbo?'] = (SUP2file['Peak coverage']>=0.95) & (SUP2file['Acepto las promos turbo?'] == 'Si')



def getPromoTracker():
  creds, _ = default()
  gc = gspread.authorize(creds)

  sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/11aWZIXrsLxZrIFQguT6E7wCz_VxvxkG0HCIr_z-l-YQ/")
  ws = sh.worksheet("New PromoTracker")


  raw = ws.get_all_values()
  headers = raw[5]
  rows = raw[6:]

  pT = pd.DataFrame(rows, columns=headers)
  return pT


def getDishesForms():
  creds, _ = default()
  gc = gspread.authorize(creds)

  sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/11aWZIXrsLxZrIFQguT6E7wCz_VxvxkG0HCIr_z-l-YQ/")
  ws2 = sh.worksheet("SSL SUP")


  raw2 = ws2.get_all_values()
  headers2 = raw2[0]
  rows2 = raw2[1:]

  forms = pd.DataFrame(rows2, columns=headers2)

  forms = forms.rename(columns={'Shop ID\n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697':'Shop ID',
                              '1. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 1',
                              '2. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 2',
                              '3. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 3',
                              '4. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 4',
                              '5. Dish ID \n\nÚnicamente 19 dígitos, sin "id_"\nEjemplo: 5764607619743089697\n\nRecuerda que los límites de precio para los items son:\nMX: $ 270 MXN\nCO: $ 43,000 COP\nCR: $ 8,000 CRC\nPE: $ 43 PEN':'Dish 5',
                              #'6. Dish ID \nUNICAMENTE PARA COLOMBIA DIDINVERSION':'Dish 6'
                              })

  return forms




In [ ]:
from datetime import date, timedelta
def getSUPsPacksDates(anio, mes):


  # obtener el primer día del mes y calcular cuántos días tiene el mes
  primer_dia = date(anio, mes, 1)
  if mes == 12:
      siguiente_mes = date(anio + 1, 1, 1)
  else:
      siguiente_mes = date(anio, mes + 1, 1)
  ultimo_dia = siguiente_mes - timedelta(days=1)
  dias_mes = (ultimo_dia - primer_dia).days + 1

  # listas de martes y viernes
  martes = []
  viernes = []

  for i in range(dias_mes):
      dia = primer_dia + timedelta(days=i)
      if dia.weekday() == 1:  # 0=lunes, 1=martes, ..., 6=domingo
          martes.append(dia)
      elif dia.weekday() == 4:
          viernes.append(dia)

  # spans de 15 días
  spans_martes = [(d, d + timedelta(days=15)) for d in martes]
  spans_viernes = [(d, d + timedelta(days=15)) for d in viernes]

  # Mostrar resultados
  print("Martes:")
  for inicio, fin in spans_martes:
      print(f"{inicio} -> {fin}")

  print("\nViernes:")
  for inicio, fin in spans_viernes:
      print(f"{inicio} -> {fin}")

getSUPsPacksDates(2025, 11)


Martes:
2025-11-04 -> 2025-11-19
2025-11-11 -> 2025-11-26
2025-11-18 -> 2025-12-03
2025-11-25 -> 2025-12-10

Viernes:
2025-11-07 -> 2025-11-22
2025-11-14 -> 2025-11-29
2025-11-21 -> 2025-12-06
2025-11-28 -> 2025-12-13


In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from gspread_dataframe import get_as_dataframe

df = get_as_dataframe(worksheet, evaluate_formulas=True, dtype=str).dropna(how="all")
df

,Shop ID,Nombre,Pais,Ciudad,Hunter,BDM,Equipo,Fecha de FO,Mes de FO,Fecha de SUP1,...,Inscripción de SUP 2 (Forms),Invitacion ELC,ID Evento (ELC),Inicio de ELC,Fin de ELC,Inscripción de ELC (Gattaran),Average daily orders,"Órdenes (mes, mes + 1)",Narvarte,SUP 3
0,5764608344996974167,Super Shawarma El Jeque,PE,Lima,josephulloa,omaramayo,BD,2025-04-01,4,2025-04-04,...,Signup,Invited,5765207997934404446,2025-05-02,2025-07-06,Signup,NaN,NaN,No,NaN
1,5764607643734510481,CHAMORROS TIJUANA,MX,Tijuana,system,Not hunter,SOB,2025-04-01,4,2025-04-04,...,Signup,Invited,5765207960747705159,2025-05-02,2025-07-06,Signup,NaN,NaN,No,NaN
2,5764608093896576492,Ahogadas rocky,MX,Guadalajara,system,Not hunter,SOB,2025-04-01,4,2025-04-04,...,Signup,Invited,5765207960747705159,2025-05-02,2025-07-06,Signup,NaN,NaN,No,NaN
3,5764608111651065183,Ensali,MX,Hermosillo,omarrendonhoyos,lballesterosaguilar,BD,2025-04-01,4,2025-04-04,...,Signup,Invited,5765207960747705159,2025-05-02,2025-07-06,Signup,NaN,NaN,No,NaN
4,5764608117225295398,Hamburguesas Ríos BS,MX,Toluca,system,Not hunter,SOB,2025-04-01,4,2025-04-04,...,No signup,No invitation,No invitation,No invitation,No invitation,No signup,NaN,NaN,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16519,5764609105172630069,Aguamanía FRUTAL,MX,Mexico City,system,Not hunter,SOB,2025-07-30,7,2025-08-01,...,No signup,No invitation,No invitation,No invitation,No invitation,No signup,NaN,NaN,No,NaN
16520,5764609143558900173,Pollos asados La cabaña,MX,Aguascalientes,system,Not hunter,SOB,2025-07-30,7,2025-08-01,...,No signup,No invitation,No invitation,No invitation,No invitation,No signup,NaN,NaN,No,NaN
16521,5764609152991890161,Las Delicias de Mamá,MX,Tuxtla Gutierrez,system,Not hunter,SOB,2025-07-31,7,2025-08-01,...,No signup,No invitation,No invitation,No invitation,No invitation,No signup,NaN,NaN,No,NaN
16522,5764609163892886021,Don Birrion,MX,Guadalajara,system,Not hunter,SOB,2025-07-31,7,2025-08-01,...,No signup,No invitation,No invitation,No invitation,No invitation,No signup,NaN,NaN,No,NaN


In [ ]:
# Asegurarte de que la columna sea datetime y mostrar las 10 fechas más recientes
print(df['Fecha de FO'].dropna().drop_duplicates().sort_values(ascending=False).head(10))

16054    2025-07-31
16053    2025-07-30
16049    2025-07-29
15484    2025-07-28
15491    2025-07-27
15497    2025-07-26
15485    2025-07-25
14907    2025-07-24
14908    2025-07-23
14916    2025-07-22
Name: Fecha de FO, dtype: object


In [ ]:
print(df.columns)

Index(['Shop ID', 'Nombre', 'Pais', 'Ciudad', 'Hunter', 'BDM', 'Equipo',
       'Fecha de FO', 'Mes de FO', 'Fecha de SUP1',
       'Inscripción de SUP 1 (Forms)', 'Fecha de SUP2',
       'Inscripción de SUP 2 (Forms)', 'Invitacion ELC', 'ID Evento (ELC)',
       'Inicio de ELC', 'Fin de ELC', 'Inscripción de ELC (Gattaran)',
       'Average daily orders', 'Órdenes (mes, mes + 1)', 'Narvarte', 'SUP 3'],
      dtype='object')


In [ ]:
#Antes excluiamos estas ciudades de las promos para el canal de Hunting/BD/Comercial (CS/TMK sí tenían promos de estas ciudades)
MX_cities_excluded = ['Aguascalientes', 'Celaya', 'Chihuahua', 'Delicias', 'Durango', 'Irapuato', 'Jaurez CHIH',
                   'Leon', 'Matamoros', 'Monterrey', 'Nuevo Laredo', 'Queretaro', 'Salamanca', 'Saltillo',
                   'San Juan del Río', 'San Luis Potosi', 'Tampico', 'Torreon', 'Victoria', 'Zacatecas']

In [ ]:
df = df[~((df['Equipo'] == 'BD') & (df['Ciudad'].isin(MX_cities_excluded)))]
df = df[df['Narvarte'] == 'No']

In [ ]:
MX_cities_excluded

['Aguascalientes',
 'Celaya',
 'Chihuahua',
 'Delicias',
 'Durango',
 'Irapuato',
 'Jaurez CHIH',
 'Leon',
 'Matamoros',
 'Monterrey',
 'Nuevo Laredo',
 'Queretaro',
 'Salamanca',
 'Saltillo',
 'San Juan del Río',
 'San Luis Potosi',
 'Tampico',
 'Torreon',
 'Victoria',
 'Zacatecas']

In [ ]:
print(df.loc[df['Equipo'] == 'BD', 'Ciudad'].unique())

['Lima' 'Hermosillo' 'Guadalajara' 'La Paz' 'Mexico City' 'Tijuana'
 'Merida' 'Mazatlan' 'Bogotá, D.C.' 'Medellín' 'Puebla' 'Culiacan'
 'Villahermosa' 'Acapulco' 'Oaxaca' 'Los Mochis-Guasave' 'Morelia'
 'Puerto Vallarta' 'Ensenada' 'Mexicali' 'Toluca' 'Colima' 'Veracruz'
 'Cordoba' 'Solidaridad' 'Cuernavaca' 'Cancun' 'Tlaxcala' 'Xalapa'
 'Tehuacan' 'Cali' 'Tepic' 'Bucaramanga' 'Coatzacoalcos' 'San Jose'
 'Barranquilla' 'Carmen' 'Tuxtla Gutierrez' 'Cúcuta' 'Cartagena'
 'Chetumal' 'Manzanillo' 'Campeche' 'Los Cabos' 'Pachuca'
 'Poza Rica de Hidalgo' 'Ciudad Obregon' 'San Cristobal de las Casas'
 'Tapachula' 'Navojoa']


In [ ]:
#Formato de fecha a columnas de tiempo
columns_date = ["Fecha de FO", "Fecha de SUP1", "Fecha de SUP2", "Inicio de ELC", "Fin de ELC"]

date_format = "%Y-%m-%d"

for col in columns_date:
  df[col] = pd.to_datetime(df[col], format = date_format, errors="coerce")

In [ ]:
#Conexión con carpeta que almacenaba los datos de Peak Hours y Avg Daily Hours para promos turbo
import os
import glob

folder_path = '/content/drive/MyDrive/qualifiers_turbo/sup2_turbo/'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

if not csv_files:
    print("❌ No se encontró ningún archivo CSV en la carpeta.")
else:
    latest_file = max(csv_files, key=os.path.getmtime)
    print(f"✅ Archivo más reciente encontrado: {latest_file}")
    peak_hours = pd.read_csv(latest_file)

peak_hours = peak_hours[['shop_id', 'dt(周)' ,'Peak coverage']]

✅ Archivo más reciente encontrado: /content/drive/MyDrive/qualifiers_turbo/sup2_turbo/103085-Peak_hours_connectivity-Peak+Coverage+During+Peak+Hours+(Both+Peaks)-20250812063001_968264.csv


In [ ]:
# Mismo proceso para ELC turbo
import os
import glob
import pandas as pd

folder_path = '/content/drive/MyDrive/qualifiers_turbo/elc_turbo/'
xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

if not xlsx_files:
    print("❌ No se encontró ningún archivo XLSX en la carpeta.")
else:
    latest_file = max(xlsx_files, key=os.path.getmtime)
    print(f"✅ Archivo más reciente encontrado: {latest_file}")

    avg_orders = pd.read_excel(latest_file)

avg_orders['avg orders'] = np.where(
    avg_orders['online_days_29'] == 0,
    0,
    avg_orders['orders_29'] / avg_orders['online_days_29']
)

avg_orders = avg_orders[['shop_id', 'avg orders']]

✅ Archivo más reciente encontrado: /content/drive/MyDrive/qualifiers_turbo/elc_turbo/ReeducionTURBO (24).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#Verificar la validez de la oferta comercial para darle las promos necesarias
spreadsheet_validity = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY/edit?gid=839382948#gid=839382948')

validity = spreadsheet_validity.worksheet('validity')
df_validity = get_as_dataframe(validity, evaluate_formulas=True, dtype=str).dropna(how="all")
df_validity.head()

,offer,year,start time,end time
0,5,2025,2025-04-28,2025-06-01
1,6,2025,2025-06-02,2025-06-29
2,7,2025,2025-06-30,2025-08-03
3,8,2025,2025-08-04,2025-08-31


In [ ]:
df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
df_validity['start time'] = pd.to_datetime(df_validity['start time'])
df_validity['end time'] = pd.to_datetime(df_validity['end time'])

In [ ]:
print(df.loc[df['Equipo'] == 'BD', 'Ciudad'].unique())

['Lima' 'Hermosillo' 'Guadalajara' 'La Paz' 'Mexico City' 'Tijuana'
 'Merida' 'Mazatlan' 'Bogotá, D.C.' 'Medellín' 'Puebla' 'Culiacan'
 'Villahermosa' 'Acapulco' 'Oaxaca' 'Los Mochis-Guasave' 'Morelia'
 'Puerto Vallarta' 'Ensenada' 'Mexicali' 'Toluca' 'Colima' 'Veracruz'
 'Cordoba' 'Solidaridad' 'Cuernavaca' 'Cancun' 'Tlaxcala' 'Xalapa'
 'Tehuacan' 'Cali' 'Tepic' 'Bucaramanga' 'Coatzacoalcos' 'San Jose'
 'Barranquilla' 'Carmen' 'Tuxtla Gutierrez' 'Cúcuta' 'Cartagena'
 'Chetumal' 'Manzanillo' 'Campeche' 'Los Cabos' 'Pachuca'
 'Poza Rica de Hidalgo' 'Ciudad Obregon' 'San Cristobal de las Casas'
 'Tapachula' 'Navojoa']


In [ ]:
# 1️⃣ Convertimos todas las fechas a datetime
df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
df_validity['start time'] = pd.to_datetime(df_validity['start time'])
df_validity['end time'] = pd.to_datetime(df_validity['end time'])

# 2️⃣ Creamos columna clave temporal para cross join
df['_key'] = 1
df_validity['_key'] = 1

# 3️⃣ Hacemos el cross join
df_comb = pd.merge(df, df_validity, on='_key')

# 4️⃣ Filtramos solo las filas donde 'Fecha de FO' está en el rango
df_comb = df_comb[
    (df_comb['Fecha de FO'] >= df_comb['start time']) &
    (df_comb['Fecha de FO'] <= df_comb['end time'])
]

# 5️⃣ Nos quedamos solo con la primera oferta por cada Fecha de FO
# Esto quita duplicados para el merge posterior
df_unique = df_comb.drop_duplicates(subset=['Fecha de FO'])

# 6️⃣ Hacemos merge para traer 'offer' de df_unique al df original
df = df.merge(
    df_unique[['Fecha de FO', 'offer']],
    on='Fecha de FO',
    how='left'
)

# 7️⃣ Quitamos la columna temporal
df = df.drop(columns=['_key'], errors='ignore')


In [ ]:
print(df.loc[df['Equipo'] == 'BD', 'Ciudad'].unique())


['Lima' 'Hermosillo' 'Guadalajara' 'La Paz' 'Mexico City' 'Tijuana'
 'Merida' 'Mazatlan' 'Bogotá, D.C.' 'Medellín' 'Puebla' 'Culiacan'
 'Villahermosa' 'Acapulco' 'Oaxaca' 'Los Mochis-Guasave' 'Morelia'
 'Puerto Vallarta' 'Ensenada' 'Mexicali' 'Toluca' 'Colima' 'Veracruz'
 'Cordoba' 'Solidaridad' 'Cuernavaca' 'Cancun' 'Tlaxcala' 'Xalapa'
 'Tehuacan' 'Cali' 'Tepic' 'Bucaramanga' 'Coatzacoalcos' 'San Jose'
 'Barranquilla' 'Carmen' 'Tuxtla Gutierrez' 'Cúcuta' 'Cartagena'
 'Chetumal' 'Manzanillo' 'Campeche' 'Los Cabos' 'Pachuca'
 'Poza Rica de Hidalgo' 'Ciudad Obregon' 'San Cristobal de las Casas'
 'Tapachula' 'Navojoa']


In [ ]:
#Ver que día es hoy
import datetime
from datetime import timedelta
today = datetime.datetime.now()
week_day = today.strftime("%A")
print("Today is:", week_day)

Today is: Tuesday


In [ ]:
#Para lunes se clasificaban por fecha de fo cada tipo de shop id
def get_tuesday_by_num_week(year, week):
  monday = datetime.datetime.strptime(f'{year}-W{week}-1', '%G-W%V-%u')
  tuesday = monday + datetime.timedelta(days=1)
  return tuesday

def monday_filter_SUP2(df):
  df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
  actual_week = datetime.datetime.now().isocalendar().week
  actual_year = datetime.datetime.now().year

  two_weeks_prior = actual_week - 2
  if two_weeks_prior <= 0:
      two_weeks_prior += 52
      actual_year -= 1

# SUP 2
  two_week_prior_tuesday = get_tuesday_by_num_week(actual_year, two_weeks_prior)
  two_week_prior_friday = two_week_prior_tuesday - datetime.timedelta(days=4)

  df_SUP2_monday = df[df['Equipo'] == 'BD']
  df_SUP2_monday = df_SUP2_monday[(df_SUP2_monday['Fecha de FO'] >= two_week_prior_friday) & (df_SUP2_monday['Fecha de FO'] < two_week_prior_tuesday)]

  columns_to_show = ["Shop ID","Pais", "Ciudad", "Fecha de FO", "Fecha de SUP2", "offer"]
  df_SUP2_monday = df_SUP2_monday[columns_to_show]

  return df_SUP2_monday

def monday_filter_ELC(df):
  df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
  actual_week = datetime.datetime.now().isocalendar().week
  actual_year = datetime.datetime.now().year

  four_weeks_prior = actual_week - 4
  if four_weeks_prior <= 0:
      four_weeks_prior += 52
      actual_year -= 1

# ELC
  four_week_prior_tuesday = get_tuesday_by_num_week(actual_year, four_weeks_prior)
  four_week_prior_friday = four_week_prior_tuesday - datetime.timedelta(days=4)

  df_ELC_monday = df[df['Equipo'] == 'BD']
  df_ELC_monday = df_ELC_monday[(df_ELC_monday['Fecha de FO'] >= four_week_prior_friday) & (df_ELC_monday['Fecha de FO'] < four_week_prior_tuesday)]

  columns_to_show = ["Shop ID","Pais", "Ciudad", "Fecha de FO", "Invitacion ELC", "offer"]
  df_ELC_monday = df_ELC_monday[columns_to_show]

  return df_ELC_monday


In [ ]:
df_monday_SUP2 = monday_filter_SUP2(df)
df_monday_ELC = monday_filter_ELC(df)

In [ ]:
print(df['Fecha de FO'].dropna().drop_duplicates().sort_values(ascending=False).head(10))

15294   2025-07-31
15293   2025-07-30
15289   2025-07-29
14751   2025-07-28
14758   2025-07-27
14764   2025-07-26
14752   2025-07-25
14198   2025-07-24
14199   2025-07-23
14207   2025-07-22
Name: Fecha de FO, dtype: datetime64[ns]


In [ ]:
#Cruce de listado de lunes con las consideracion de Peak Hours (SUP2) y de Avg Daily Orders (ELC) para promos TURBO
import pandas as pd

# 1. Convertir la fecha (inicio de la semana)
peak_hours['start_date'] = pd.to_datetime(peak_hours['dt(周)'].str.split('/').str[0])

# 2. Limpiar y convertir 'Peak coverage' de porcentaje a decimal
peak_hours['Peak coverage'] = (
    peak_hours['Peak coverage']
    .astype(str)
    .str.replace('%', '', regex=False)
    .astype(float) / 100
)

# 3. Ordenar por shop_id y fecha (de más reciente a más antigua)
peak_hours = peak_hours.sort_values(['shop_id', 'start_date'], ascending=[True, False])

# 4. Conservar solo los 2 registros más recientes por shop_id
peak_hours = peak_hours.groupby('shop_id').head(2)

# 5. Calcular el promedio de 'Peak coverage' por shop_id
peak_hours = (
    peak_hours.groupby('shop_id', as_index=False)['Peak coverage']
    .mean()
)

# 6. (Explícito) Eliminar cualquier duplicado por seguridad (aunque no debería haber)
peak_hours = peak_hours.drop_duplicates(subset='shop_id')


In [ ]:
df_monday_SUP2['Shop ID'] = df_monday_SUP2['Shop ID'].astype(str)
peak_hours['shop_id'] = peak_hours['shop_id'].astype(str)

# Asegurar unicidad en peak_hours
peak_hours = peak_hours.drop_duplicates(subset='shop_id')

# Hacer el merge
df_monday_SUP2 = df_monday_SUP2.merge(
    peak_hours[['shop_id', 'Peak coverage']],
    left_on='Shop ID',
    right_on='shop_id',
    how='left'
).drop(columns=['shop_id'])

# Rellenar nulos con 0
df_monday_SUP2['Peak coverage'] = df_monday_SUP2['Peak coverage'].fillna(0)


In [ ]:
#Validación de aceptación de orders
spreadsheet_accepeted = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/11aWZIXrsLxZrIFQguT6E7wCz_VxvxkG0HCIr_z-l-YQ/edit?gid=866635236#gid=866635236'
)

accepted = spreadsheet_accepeted.worksheet('SSL SUP')
df_accepted = get_as_dataframe(accepted, evaluate_formulas=True, dtype=str).dropna(how="all")

# Renombrar columnas
df_accepted.columns.values[7] = 'shop_id'
df_accepted.columns.values[14] = 'accepted'

# Quedarse solo con columnas relevantes
df_accepted = df_accepted[['shop_id', 'accepted']]

# Normalizar y mapear valores
valores_si = ['sí', 'si', 'Sí', 'Si', '1']
df_accepted['accepted'] = df_accepted['accepted'].astype(str).str.strip().str.lower().apply(
    lambda x: 'Sí' if x in ['sí', 'si', '1'] else 'No'
)

df_accepted = df_accepted.drop_duplicates(subset='shop_id', keep='last')

# Asegurar que ambos dataframes tengan shop_id como string
df_monday_SUP2['Shop ID'] = df_monday_SUP2['Shop ID'].astype(str)
df_accepted['shop_id'] = df_accepted['shop_id'].astype(str)




In [ ]:
#Merge para saber si aceptaron o no
df_monday_SUP2 = df_monday_SUP2.merge(
    df_accepted[['shop_id', 'accepted']],
    left_on='Shop ID',
    right_on='shop_id',
    how='left'
).drop(columns=['shop_id']).assign(accepted=lambda x: x['accepted'].fillna('No'))

In [ ]:
#Merge para saber si cumplió requisitos de ELC TURBO
df_monday_ELC['Shop ID'] = df_monday_ELC['Shop ID'].astype(str)
avg_orders['shop_id'] = avg_orders['shop_id'].astype(str)

df_monday_ELC = df_monday_ELC.merge(
    avg_orders[['shop_id', 'avg orders']],
    left_on='Shop ID',
    right_on='shop_id',
    how='left'
).drop(columns=['shop_id'])

df_monday_ELC['avg orders'] = df_monday_ELC['avg orders'].fillna(0)

In [ ]:
# 1️⃣ Convertir y limpiar 'Peak coverage'
df_monday_SUP2['Peak coverage'] = pd.to_numeric(df_monday_SUP2['Peak coverage'], errors='coerce')
df_monday_SUP2['Peak coverage'] = df_monday_SUP2['Peak coverage'].fillna(0)

# 2️⃣ Asegurar que 'offer' sea string limpio
df_monday_SUP2['offer'] = df_monday_SUP2['offer'].astype(str).str.strip()

# 3️⃣ Definir condición turbo basada en país y cobertura
cond_turbo_SUP2 = (
    (
        (df_monday_SUP2['Pais'].isin(['MX', 'CO']) & (df_monday_SUP2['Peak coverage'] >= 0.89)) |
        (df_monday_SUP2['Pais'].isin(['PE', 'CR']) & (df_monday_SUP2['Peak coverage'] >= 0.94))
    ) &
    (df_monday_SUP2['accepted'] == 'Sí')
)

# 4️⃣ Crear diccionario para guardar los DataFrames segmentados
dfs_segmentados = {}

for offer in df_monday_SUP2['offer'].unique():
    df_offer = df_monday_SUP2[df_monday_SUP2['offer'] == offer]

    # turbo
    df_turbo = df_offer[cond_turbo_SUP2.loc[df_offer.index]].copy()
    nombre_turbo = f"monday_SUP2_turbo_{offer}"
    globals()[nombre_turbo] = df_turbo.copy()  # Asigna como variable global

    # normal
    df_normal = df_offer[~cond_turbo_SUP2.loc[df_offer.index]].copy()
    nombre_normal = f"monday_SUP2_normal_{offer}"
    globals()[nombre_normal] = df_normal.copy()  # Asigna como variable global

    print(f"✅ Variables creadas: {nombre_turbo} ({len(df_turbo)} filas), {nombre_normal} ({len(df_normal)} filas)")

✅ Variables creadas: monday_SUP2_turbo_7 (6 filas), monday_SUP2_normal_7 (68 filas)


In [ ]:
#Conexión con el calendario de ELC para saber fecha de fin de promo
spreadsheet_elc = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY/edit?gid=1534681415#gid=1534681415')

elc_calendar = spreadsheet_elc.worksheet('elc')
df_elc_calendar = get_as_dataframe(elc_calendar, evaluate_formulas=True, dtype=str).dropna(how="all")


In [ ]:
import pandas as pd

# Asegúrate que las columnas de fecha sean datetime
df_monday_ELC['Fecha de FO'] = pd.to_datetime(df_monday_ELC['Fecha de FO'], errors='coerce')
df_elc_calendar['FO Start Date'] = pd.to_datetime(df_elc_calendar['FO Start Date'], errors='coerce')
df_elc_calendar['FO End Date'] = pd.to_datetime(df_elc_calendar['FO End Date'], errors='coerce')
df_elc_calendar['ELC End Date'] = pd.to_datetime(df_elc_calendar['ELC End Date'], errors='coerce')

# Función para buscar el ELC End Date correspondiente al rango donde cae Fecha de FO
def buscar_ELC(fecha_fo):
    # Filtrar filas en df_elc_calendar donde fecha_fo está dentro del rango [FO Start Date, FO End Date]
    df_filtrado = df_elc_calendar[(df_elc_calendar['FO Start Date'] <= fecha_fo) & (fecha_fo <= df_elc_calendar['FO End Date'])]
    if not df_filtrado.empty:
        # Si hay más de uno, puedes decidir qué devolver (por ejemplo el primero)
        return df_filtrado.iloc[0]['ELC End Date']
    else:
        return pd.NaT  # o None

# Aplicar la función fila por fila en df_monday_ELC y crear la columna nueva
df_monday_ELC['ELC End Date'] = df_monday_ELC['Fecha de FO'].apply(buscar_ELC)

In [ ]:
print(df.loc[df['Equipo'] == 'BD', 'Ciudad'].unique())


['Lima' 'Hermosillo' 'Guadalajara' 'La Paz' 'Mexico City' 'Tijuana'
 'Merida' 'Mazatlan' 'Bogotá, D.C.' 'Medellín' 'Puebla' 'Culiacan'
 'Villahermosa' 'Acapulco' 'Oaxaca' 'Los Mochis-Guasave' 'Morelia'
 'Puerto Vallarta' 'Ensenada' 'Mexicali' 'Toluca' 'Colima' 'Veracruz'
 'Cordoba' 'Solidaridad' 'Cuernavaca' 'Cancun' 'Tlaxcala' 'Xalapa'
 'Tehuacan' 'Cali' 'Tepic' 'Bucaramanga' 'Coatzacoalcos' 'San Jose'
 'Barranquilla' 'Carmen' 'Tuxtla Gutierrez' 'Cúcuta' 'Cartagena'
 'Chetumal' 'Manzanillo' 'Campeche' 'Los Cabos' 'Pachuca'
 'Poza Rica de Hidalgo' 'Ciudad Obregon' 'San Cristobal de las Casas'
 'Tapachula' 'Navojoa']


In [ ]:
import pandas as pd

# --- Asegurar que 'offer' esté limpio ---
df_monday_ELC['offer'] = df_monday_ELC['offer'].astype(str).str.strip()

# --- Condición para turbo ---
cond_turbo_ELC = df_monday_ELC['avg orders'] >= 5

# --- Diccionario para guardar los DataFrames segmentados ---
dfs_segmentados = {}

# --- Recorrer cada tipo de offer ---
for offer in df_monday_ELC['offer'].unique():
    df_offer = df_monday_ELC[df_monday_ELC['offer'] == offer]

    # turbo
    df_turbo = df_offer[cond_turbo_ELC.loc[df_offer.index]].copy()
    nombre_base_turbo = f"monday_ELC_turbo_{offer}"
    globals()[nombre_base_turbo] = df_turbo.copy()
    dfs_segmentados[nombre_base_turbo] = {}

    # listas para impresión
    impresiones_creadas = []
    impresiones_check = []

    for fecha in df_turbo['ELC End Date'].dropna().unique():
        df_sub = df_turbo[df_turbo['ELC End Date'] == fecha]
        subnombre = f"{nombre_base_turbo}_{fecha.strftime('%Y-%m-%d')}"
        globals()[subnombre] = df_sub.copy()
        dfs_segmentados[nombre_base_turbo][fecha] = df_sub

        impresiones_creadas.append(f"📁 Creado: {subnombre} ({len(df_sub)} filas)")
        impresiones_check.append(f"✅ Variables creadas: {subnombre} ({len(df_sub)} filas)")

    # normal
    df_normal = df_offer[~cond_turbo_ELC.loc[df_offer.index]].copy()
    nombre_base_normal = f"monday_ELC_normal_{offer}"
    globals()[nombre_base_normal] = df_normal.copy()
    dfs_segmentados[nombre_base_normal] = {}

    for fecha in df_normal['ELC End Date'].dropna().unique():
        df_sub = df_normal[df_normal['ELC End Date'] == fecha]
        subnombre = f"{nombre_base_normal}_{fecha.strftime('%Y-%m-%d')}"
        globals()[subnombre] = df_sub.copy()
        dfs_segmentados[nombre_base_normal][fecha] = df_sub

        impresiones_creadas.append(f"📁 Creado: {subnombre} ({len(df_sub)} filas)")
        impresiones_check.append(f"✅ Variables creadas: {subnombre} ({len(df_sub)} filas)")

    # --- Imprimir agrupado ---
    for linea in impresiones_creadas:
        print(linea)
    for linea in impresiones_check:
        print(linea)


📁 Creado: monday_ELC_turbo_7_2025-09-28 (13 filas)
📁 Creado: monday_ELC_normal_7_2025-09-28 (73 filas)
✅ Variables creadas: monday_ELC_turbo_7_2025-09-28 (13 filas)
✅ Variables creadas: monday_ELC_normal_7_2025-09-28 (73 filas)


In [ ]:
def get_monday_by_num_week(year, week):
  monday = datetime.datetime.strptime(f'{year}-W{week}-1', '%G-W%V-%u')
  return monday

def tuesday_filter_SUP1(df):
  df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
  actual_week = datetime.datetime.now().isocalendar().week
  actual_year = datetime.datetime.now().year

  one_weeks_prior = actual_week - 1
  if one_weeks_prior <= 0:
      one_weeks_prior += 52
      actual_year -= 1

# SUP 1
  monday = get_monday_by_num_week(actual_year, actual_week)
  tuesday = monday + datetime.timedelta(days=1)
  friday = monday - datetime.timedelta(days=3)

  df_SUP1_tuesday = df
  df_SUP1_tuesday = df[df['Equipo'] == 'BD']
  df_SUP1_tuesday = df_SUP1_tuesday[(df_SUP1_tuesday['Fecha de FO'] >= friday) & (df_SUP1_tuesday['Fecha de FO'] < tuesday )]

  columns_to_show = ["Shop ID","Pais", "Ciudad", "Fecha de FO", "Fecha de SUP1", "offer"]
  df_SUP1_tuesday = df_SUP1_tuesday[columns_to_show]

  return df_SUP1_tuesday

In [ ]:
print(df['Fecha de FO'].dropna().drop_duplicates().sort_values(ascending=False).head(10))

15294   2025-07-31
15293   2025-07-30
15289   2025-07-29
14751   2025-07-28
14758   2025-07-27
14764   2025-07-26
14752   2025-07-25
14198   2025-07-24
14199   2025-07-23
14207   2025-07-22
Name: Fecha de FO, dtype: datetime64[ns]


In [ ]:
df_tuesday = tuesday_filter_SUP1(df)

In [ ]:
# Nos aseguramos que offer esté como texto o número limpio
df_tuesday['offer'] = df_tuesday['offer'].astype(str).str.strip()

# Verificamos los valores únicos
ofertas_unicas = df_tuesday['offer'].dropna().unique()

dfs_por_oferta = {}  # aquí guardaremos los nuevos DataFrames

if len(ofertas_unicas) == 1:
    numero = ofertas_unicas[0]
    nombre_df = f"df_tuesday_{numero}"
    dfs_por_oferta[nombre_df] = df_tuesday

    # Creamos la variable dinámica
    globals()[nombre_df] = df_tuesday

    print(f"✅ Solo hay un valor en offer: {numero}. Se guarda y crea variable: {nombre_df}")

else:
    # Hay varios números → creamos un df por cada número
    for numero in ofertas_unicas:
        df_filtrado = df_tuesday[df_tuesday['offer'] == numero].copy()
        nombre_df = f"tuesday_SUP1_{numero}"
        dfs_por_oferta[nombre_df] = df_filtrado

        # Creamos la variable dinámica
        globals()[nombre_df] = df_filtrado

        print(f"✅ Creado y variable lista: {nombre_df} con {len(df_filtrado)} filas.")


if len(ofertas_unicas) == 1:
    numero = ofertas_unicas[0]
    nombre_df = f"tuesday_SUP1_{numero}"  # 👈 Aquí el cambio
    dfs_por_oferta[nombre_df] = df_tuesday

    # Creamos la variable dinámica
    globals()[nombre_df] = df_tuesday

    print(f"✅ Solo hay un valor en offer: {numero}. Se guarda y crea variable: {nombre_df}")


In [ ]:
#Generar listados para INBOUND y para BD, de SUP2 y de ELC (TURBO Y NORMAL)
def get_thursday_by_num_week(year, week):
  monday = datetime.datetime.strptime(f'{year}-W{week}-1', '%G-W%V-%u')
  thursday = monday + datetime.timedelta(days=3)
  return thursday

def thursday_filter_SUP2(df):
  df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
  actual_week = datetime.datetime.now().isocalendar().week
  actual_year = datetime.datetime.now().year

  two_weeks_prior = actual_week - 2
  if two_weeks_prior <= 0:
      two_weeks_prior += 52
      actual_year -= 1

# SUP 2
  two_week_prior_thursday = get_thursday_by_num_week(actual_year, two_weeks_prior)
  two_week_prior_tuesday = two_week_prior_thursday - datetime.timedelta(days=2)
  two_week_prior_friday = two_week_prior_thursday - datetime.timedelta(days=6)

  df_SUP2_thursday_BD = df[df['Equipo'] == 'BD']
  df_SUP2_thursday_BD = df_SUP2_thursday_BD[(df_SUP2_thursday_BD['Fecha de FO'] >= two_week_prior_tuesday) & (df_SUP2_thursday_BD['Fecha de FO'] <= two_week_prior_thursday)]

  df_SUP2_thursday_Inbound = df[df['Equipo'] == 'INBOUND']
  df_SUP2_thursday_Inbound = df_SUP2_thursday_Inbound[(df_SUP2_thursday_Inbound['Fecha de FO'] >= two_week_prior_friday) & (df_SUP2_thursday_Inbound['Fecha de FO'] <= two_week_prior_thursday)]

  columns_to_show = ["Shop ID","Pais", "Ciudad", "Fecha de FO", "Fecha de SUP2", 'offer']
  df_SUP2_thursday_BD = df_SUP2_thursday_BD[columns_to_show]
  df_SUP2_thursday_Inbound = df_SUP2_thursday_Inbound[columns_to_show]
  df_SUP2_thursday = pd.concat([df_SUP2_thursday_BD, df_SUP2_thursday_Inbound])

  return df_SUP2_thursday

def thursday_filter_ELC(df):
  df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
  actual_week = datetime.datetime.now().isocalendar().week
  actual_year = datetime.datetime.now().year

  four_weeks_prior = actual_week - 4
  if four_weeks_prior <= 0:
      four_weeks_prior += 52
      actual_year -= 1

# ELC
  four_week_prior_thursday = get_thursday_by_num_week(actual_year, four_weeks_prior)
  four_week_prior_tuesday = four_week_prior_thursday - datetime.timedelta(days=2)
  four_week_prior_friday = four_week_prior_thursday - datetime.timedelta(days=6)

  df_ELC_thursday_BD = df[df['Equipo'] == 'BD']
  df_ELC_thursday_BD = df_ELC_thursday_BD[(df_ELC_thursday_BD['Fecha de FO'] >= four_week_prior_tuesday) & (df_ELC_thursday_BD['Fecha de FO'] <= four_week_prior_thursday)]

  df_ELC_thursday_Inbound = df[df['Equipo'] == 'INBOUND']
  df_ELC_thursday_Inbound = df_ELC_thursday_Inbound[(df_ELC_thursday_Inbound['Fecha de FO'] >= four_week_prior_friday) & (df_ELC_thursday_Inbound['Fecha de FO'] <= four_week_prior_thursday)]

  columns_to_show = ["Shop ID","Pais", "Ciudad", "Fecha de FO", "Fecha de SUP2", 'offer']
  df_ELC_thursday_BD = df_ELC_thursday_BD[columns_to_show]
  df_ELC_thursday_Inbound = df_ELC_thursday_Inbound[columns_to_show]
  df_ELC_thursday = pd.concat([df_ELC_thursday_BD, df_ELC_thursday_Inbound])

  return df_ELC_thursday


In [ ]:
#Aplicamos las funciones a estos 2 dfs
df_thursday_SUP2 = thursday_filter_SUP2(df)
df_thursday_ELC = thursday_filter_ELC(df)

In [ ]:
#Merge de Peak hours
df_thursday_SUP2['Shop ID'] = df_thursday_SUP2['Shop ID'].astype(str)
peak_hours['shop_id'] = peak_hours['shop_id'].astype(str)

df_thursday_SUP2 = df_thursday_SUP2.merge(
    peak_hours[['shop_id', 'Peak coverage']],
    left_on='Shop ID',    # columna en df_thursday_SUP2
    right_on='shop_id',  # columna en peak
    how='left'
).drop(columns=['shop_id'])

df_thursday_SUP2['Peak coverage'] = df_thursday_SUP2['Peak coverage'].fillna(0)

In [ ]:
# Merge de si aceptaron promo turbo o no de SUP2
df_thursday_SUP2 = df_thursday_SUP2.merge(
    df_accepted[['shop_id', 'accepted']],
    left_on='Shop ID',    # columna en df_thursday_SUP2
    right_on='shop_id',  # columna en peak
    how='left'
).drop(columns=['shop_id'])

In [ ]:
# Verificar condiciones de turbo
df_thursday_SUP2['Peak coverage'] = pd.to_numeric(df_thursday_SUP2['Peak coverage'], errors='coerce')
df_thursday_SUP2['Peak coverage'] = df_thursday_SUP2['Peak coverage'].fillna(0)

cond_turbo_sup2 = (
    (
        (df_thursday_SUP2['Pais'].isin(['MX', 'CO']) & (df_thursday_SUP2['Peak coverage'] >= 0.89)) |
        (df_thursday_SUP2['Pais'].isin(['PE', 'CR']) & (df_thursday_SUP2['Peak coverage'] >= 0.94))
    ) &
    (df_thursday_SUP2['accepted'] == 'Sí')
)

# 4️⃣ Crear diccionario para guardar los DataFrames segmentados
dfs_segmentados = {}

for offer in df_thursday_SUP2['offer'].unique():
    df_offer = df_thursday_SUP2[df_thursday_SUP2['offer'] == offer]

    # turbo
    df_turbo = df_offer[cond_turbo_sup2.loc[df_offer.index]].copy()
    nombre_turbo = f"thursday_SUP2_turbo_{offer}"
    globals()[nombre_turbo] = df_turbo.copy()  # Asigna como variable global

    # normal
    df_normal = df_offer[~cond_turbo_sup2.loc[df_offer.index]].copy()
    nombre_normal = f"thursday_SUP2_normal_{offer}"
    globals()[nombre_normal] = df_normal.copy()  # Asigna como variable global

    print(f"✅ Variables creadas: {nombre_turbo} ({len(df_turbo)} filas), {nombre_normal} ({len(df_normal)} filas)")


✅ Variables creadas: thursday_SUP2_turbo_7 (5 filas), thursday_SUP2_normal_7 (54 filas)


In [ ]:
df_thursday_ELC['Shop ID'] = df_thursday_ELC['Shop ID'].astype(str)
avg_orders['shop_id'] = avg_orders['shop_id'].astype(str)

df_thursday_ELC = df_thursday_ELC.merge(
    avg_orders[['shop_id', 'avg orders']],
    left_on='Shop ID',    # columna en df_thursday_ELC
    right_on='shop_id',  # columna en peak
    how='left'
).drop(columns=['shop_id'])

df_thursday_ELC['avg orders'] = df_thursday_ELC['avg orders'].fillna(0)

In [ ]:
import pandas as pd

# Asegúrate que las columnas de fecha sean datetime
df_thursday_ELC['Fecha de FO'] = pd.to_datetime(df_thursday_ELC['Fecha de FO'], errors='coerce')
df_elc_calendar['FO Start Date'] = pd.to_datetime(df_elc_calendar['FO Start Date'], errors='coerce')
df_elc_calendar['FO End Date'] = pd.to_datetime(df_elc_calendar['FO End Date'], errors='coerce')
df_elc_calendar['ELC End Date'] = pd.to_datetime(df_elc_calendar['ELC End Date'], errors='coerce')

# Función para buscar el ELC End Date correspondiente al rango donde cae Fecha de FO
def buscar_ELC(fecha_fo):
    # Filtrar filas en df_elc_calendar donde fecha_fo está dentro del rango [FO Start Date, FO End Date]
    df_filtrado = df_elc_calendar[(df_elc_calendar['FO Start Date'] <= fecha_fo) & (fecha_fo <= df_elc_calendar['FO End Date'])]
    if not df_filtrado.empty:
        # Si hay más de uno, puedes decidir qué devolver (por ejemplo el primero)
        return df_filtrado.iloc[0]['ELC End Date']
    else:
        return pd.NaT  # o None

# Aplicar la función fila por fila en df_thursday_ELC y crear la columna nueva
df_thursday_ELC['ELC End Date'] = df_thursday_ELC['Fecha de FO'].apply(buscar_ELC)

In [ ]:
import pandas as pd

# --- Asegurar que 'offer' esté limpio ---
df_thursday_ELC['offer'] = df_thursday_ELC['offer'].astype(str).str.strip()

# --- Condición para turbo ---
cond_turbo_ELC = df_thursday_ELC['avg orders'] >= 5

# --- Diccionario para guardar los DataFrames segmentados ---
dfs_segmentados = {}

# --- Recorrer cada tipo de offer ---
for offer in df_thursday_ELC['offer'].unique():
    df_offer = df_thursday_ELC[df_thursday_ELC['offer'] == offer]

    # turbo
    df_turbo = df_offer[cond_turbo_ELC.loc[df_offer.index]].copy()
    nombre_base_turbo = f"thursday_ELC_turbo_{offer}"
    globals()[nombre_base_turbo] = df_turbo.copy()
    dfs_segmentados[nombre_base_turbo] = {}

    # listas para impresión
    impresiones_creadas = []
    impresiones_check = []

    for fecha in df_turbo['ELC End Date'].dropna().unique():
        df_sub = df_turbo[df_turbo['ELC End Date'] == fecha]
        subnombre = f"{nombre_base_turbo}_{fecha.strftime('%Y-%m-%d')}"
        globals()[subnombre] = df_sub.copy()
        dfs_segmentados[nombre_base_turbo][fecha] = df_sub

        impresiones_creadas.append(f"📁 Creado: {subnombre} ({len(df_sub)} filas)")
        impresiones_check.append(f"✅ Variables creadas: {subnombre} ({len(df_sub)} filas)")

    # normal
    df_normal = df_offer[~cond_turbo_ELC.loc[df_offer.index]].copy()
    nombre_base_normal = f"thursday_ELC_normal_{offer}"
    globals()[nombre_base_normal] = df_normal.copy()
    dfs_segmentados[nombre_base_normal] = {}

    for fecha in df_normal['ELC End Date'].dropna().unique():
        df_sub = df_normal[df_normal['ELC End Date'] == fecha]
        subnombre = f"{nombre_base_normal}_{fecha.strftime('%Y-%m-%d')}"
        globals()[subnombre] = df_sub.copy()
        dfs_segmentados[nombre_base_normal][fecha] = df_sub

        impresiones_creadas.append(f"📁 Creado: {subnombre} ({len(df_sub)} filas)")
        impresiones_check.append(f"✅ Variables creadas: {subnombre} ({len(df_sub)} filas)")

    # --- Imprimir agrupado ---
    for linea in impresiones_creadas:
        print(linea)
    for linea in impresiones_check:
        print(linea)

📁 Creado: thursday_ELC_turbo_7_2025-09-28 (9 filas)
📁 Creado: thursday_ELC_normal_7_2025-09-28 (132 filas)
✅ Variables creadas: thursday_ELC_turbo_7_2025-09-28 (9 filas)
✅ Variables creadas: thursday_ELC_normal_7_2025-09-28 (132 filas)


In [ ]:
print(df.loc[df['Equipo'] == 'BD', 'Ciudad'].unique())

['Lima' 'Hermosillo' 'Guadalajara' 'La Paz' 'Mexico City' 'Tijuana'
 'Merida' 'Mazatlan' 'Bogotá, D.C.' 'Medellín' 'Puebla' 'Culiacan'
 'Villahermosa' 'Acapulco' 'Oaxaca' 'Los Mochis-Guasave' 'Morelia'
 'Puerto Vallarta' 'Ensenada' 'Mexicali' 'Toluca' 'Colima' 'Veracruz'
 'Cordoba' 'Solidaridad' 'Cuernavaca' 'Cancun' 'Tlaxcala' 'Xalapa'
 'Tehuacan' 'Cali' 'Tepic' 'Bucaramanga' 'Coatzacoalcos' 'San Jose'
 'Barranquilla' 'Carmen' 'Tuxtla Gutierrez' 'Cúcuta' 'Cartagena'
 'Chetumal' 'Manzanillo' 'Campeche' 'Los Cabos' 'Pachuca'
 'Poza Rica de Hidalgo' 'Ciudad Obregon' 'San Cristobal de las Casas'
 'Tapachula' 'Navojoa']


In [ ]:
def get_friday_by_num_week(year, week):
  monday = datetime.datetime.strptime(f'{year}-W{week}-1', '%G-W%V-%u')
  friday = monday + datetime.timedelta(days=3)
  return friday

def friday_filter_SUP1(df):
  df['Fecha de FO'] = pd.to_datetime(df['Fecha de FO'])
  actual_week = datetime.datetime.now().isocalendar().week
  actual_year = datetime.datetime.now().year

  monday = get_monday_by_num_week(actual_year, actual_week)
  tuesday = monday + datetime.timedelta(days=1)
  thursday = monday + datetime.timedelta(days=3)
  friday = monday - datetime.timedelta(days=3)

  df_SUP1_friday_BD = df[df['Equipo'] == 'BD']
  df_SUP1_friday_BD = df_SUP1_friday_BD[(df_SUP1_friday_BD['Fecha de FO'] >= tuesday) & (df_SUP1_friday_BD['Fecha de FO'] <= thursday)]

  df_SUP1_friday_Inbound = df[df['Equipo'] == 'INBOUND']
  df_SUP1_friday_Inbound = df_SUP1_friday_Inbound[(df_SUP1_friday_Inbound['Fecha de FO'] >= friday) & (df_SUP1_friday_Inbound['Fecha de FO'] <= thursday)]

  columns_to_show = ["Shop ID","Pais", "Ciudad", "Fecha de FO", "Fecha de SUP1", 'offer']
  df_SUP1_friday_BD = df_SUP1_friday_BD[columns_to_show]
  df_SUP1_friday_Inbound = df_SUP1_friday_Inbound[columns_to_show]
  df_SUP1_friday = pd.concat([df_SUP1_friday_BD, df_SUP1_friday_Inbound])

  return df_SUP1_friday

In [ ]:
df_friday = friday_filter_SUP1(df)

In [ ]:
df_friday

,Shop ID,Pais,Ciudad,Fecha de FO,Fecha de SUP1,offer


In [ ]:
# Nos aseguramos que offer esté como texto o número limpio
df_friday['offer'] = df_friday['offer'].astype(str).str.strip()

# Verificamos los valores únicos
ofertas_unicas = df_friday['offer'].dropna().unique()

dfs_por_oferta = {}  # aquí guardaremos los nuevos DataFrames

if len(ofertas_unicas) == 1:
    numero = ofertas_unicas[0]
    nombre_df = f"friday_SUP1_{numero}"
    dfs_por_oferta[nombre_df] = df_friday

    # Creamos la variable dinámica
    globals()[nombre_df] = df_friday

    print(f"✅ Solo hay un valor en offer: {numero}. Se guarda y crea variable: {nombre_df}")

else:
    # Hay varios números → creamos un df por cada número
    for numero in ofertas_unicas:
        df_filtrado = df_friday[df_friday['offer'] == numero].copy()
        nombre_df = f"df_friday_{numero}"
        dfs_por_oferta[nombre_df] = df_filtrado

        # Creamos la variable dinámica
        globals()[nombre_df] = df_filtrado

        print(f"✅ Creado y variable lista: {nombre_df} con {len(df_filtrado)} filas.")

In [ ]:
# Lista de prefijos
prefixes = [
    "monday_SUP2_normal_",
    "monday_SUP2_turbo_",
    "monday_ELC_normal_",
    "monday_ELC_turbo_",
    "tuesday_SUP1_",
    "thursday_SUP2_normal_",
    "thursday_SUP2_turbo_",
    "thursday_ELC_normal_",
    "thursday_ELC_turbo_",
    "friday_SUP1_"
]

# Rango de números que usas al final del nombre de los DataFrames
start_num = 1
end_num = 12  # o el número que necesites

GENERACION DE LISTADOS

In [ ]:
import re

fecha_pattern = re.compile(r'.*ELC.*_\d{4}-\d{2}-\d{2}$')

dfs_dict = {}

for prefix in prefixes:
    for i in range(start_num, end_num):
        base_name = f"{prefix}{i}"
        encontrados = [var for var in globals() if var.startswith(base_name)]
        for var_name in encontrados:
            # Filtrado:
            if 'ELC' in var_name:
                if fecha_pattern.match(var_name):
                    dfs_dict[var_name] = globals()[var_name]
                    print(f"✔️ Encontrado (ELC con fecha): {var_name} ({dfs_dict[var_name].shape})")
                else:
                    # Omitir los que tienen 'ELC' pero no fecha al final
                    continue
            else:
                # Sin 'ELC' siempre agregar
                dfs_dict[var_name] = globals()[var_name]
                print(f"✔️ Encontrado: {var_name} ({dfs_dict[var_name].shape})")

dataframes = list(dfs_dict.values())
print(f"\n✅ Total de dataframes encontrados: {len(dataframes)}")


✔️ Encontrado: monday_SUP2_normal_7 ((68, 8))
✔️ Encontrado: monday_SUP2_turbo_7 ((6, 8))
✔️ Encontrado (ELC con fecha): monday_ELC_normal_7_2025-09-28 ((73, 8))
✔️ Encontrado (ELC con fecha): monday_ELC_turbo_7_2025-09-28 ((13, 8))
✔️ Encontrado: thursday_SUP2_normal_7 ((54, 8))
✔️ Encontrado: thursday_SUP2_turbo_7 ((5, 8))
✔️ Encontrado (ELC con fecha): thursday_ELC_normal_7_2025-09-28 ((132, 8))
✔️ Encontrado (ELC con fecha): thursday_ELC_turbo_7_2025-09-28 ((9, 8))

✅ Total de dataframes encontrados: 8


In [ ]:
#Segmentación por paises
import re

dfs_por_pais = {}

# Regex para detectar sufijo de fecha al final del nombre
fecha_pattern = re.compile(r'.*ELC.*_\d{4}-\d{2}-\d{2}$')

for original_name, df in dfs_dict.items():
    # Filtrar: si tiene 'ELC', solo procesar si termina con fecha
    if "ELC" in original_name and not fecha_pattern.match(original_name):
        # Omitir estos
        continue

    if "Pais" not in df.columns:
        print(f"⚠️ El DataFrame '{original_name}' no tiene la columna 'Pais'. Se omite.")
        continue

    for pais in df["Pais"].dropna().unique():
        df_pais = df[df["Pais"] == pais].copy()

        # Mantener solo la primera columna y renombrarla a '0'
        primera_col = df_pais.columns[0]
        df_pais = df_pais[[primera_col]].copy()
        df_pais.columns = [0]

        nombre_nuevo = f"{pais}_{original_name}"
        dfs_por_pais[nombre_nuevo] = df_pais
        globals()[nombre_nuevo] = df_pais  # opcional: crear variable global
        print(f"🌎 Creado: {nombre_nuevo} ({len(df_pais)} filas, columnas: {df_pais.columns.tolist()})")


🌎 Creado: PE_monday_SUP2_normal_7 (4 filas, columnas: [0])
🌎 Creado: CO_monday_SUP2_normal_7 (9 filas, columnas: [0])
🌎 Creado: CR_monday_SUP2_normal_7 (2 filas, columnas: [0])
🌎 Creado: MX_monday_SUP2_normal_7 (53 filas, columnas: [0])
🌎 Creado: MX_monday_SUP2_turbo_7 (5 filas, columnas: [0])
🌎 Creado: CO_monday_SUP2_turbo_7 (1 filas, columnas: [0])
🌎 Creado: MX_monday_ELC_normal_7_2025-09-28 (41 filas, columnas: [0])
🌎 Creado: CO_monday_ELC_normal_7_2025-09-28 (25 filas, columnas: [0])
🌎 Creado: PE_monday_ELC_normal_7_2025-09-28 (5 filas, columnas: [0])
🌎 Creado: CR_monday_ELC_normal_7_2025-09-28 (2 filas, columnas: [0])
🌎 Creado: MX_monday_ELC_turbo_7_2025-09-28 (10 filas, columnas: [0])
🌎 Creado: PE_monday_ELC_turbo_7_2025-09-28 (1 filas, columnas: [0])
🌎 Creado: CO_monday_ELC_turbo_7_2025-09-28 (2 filas, columnas: [0])
🌎 Creado: CO_thursday_SUP2_normal_7 (18 filas, columnas: [0])
🌎 Creado: CR_thursday_SUP2_normal_7 (1 filas, columnas: [0])
🌎 Creado: MX_thursday_SUP2_normal_7 (35 f

In [ ]:
import os

# Pedir al usuario que indique el día de la semana
dia_seleccionado = input("🔎 Ingresa el día que deseas subir (ej. monday, tuesday, etc.): ").strip().lower()

# Carpeta destino en tu Drive (modifica si es necesario)
carpeta_drive = "/content/drive/MyDrive/Events creation"
os.makedirs(carpeta_drive, exist_ok=True)

# Guardar solo los DataFrames que coinciden con el día seleccionado
conteo = 0
for name, df in dfs_por_pais.items():
    if dia_seleccionado in name.lower():
        archivo = os.path.join(carpeta_drive, f"{name}.csv")
        df.to_csv(archivo, index=False)
        print(f"📁 Guardado en Drive: {archivo}")
        conteo += 1

if conteo == 0:
    print(f"⚠️ No se encontraron archivos para '{dia_seleccionado}'.")
else:
    print(f"\n✅ Total de archivos guardados para '{dia_seleccionado}': {conteo}")


🔎 Ingresa el día que deseas subir (ej. monday, tuesday, etc.): monday
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/PE_monday_SUP2_normal_7.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/CO_monday_SUP2_normal_7.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/CR_monday_SUP2_normal_7.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/MX_monday_SUP2_normal_7.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/MX_monday_SUP2_turbo_7.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/CO_monday_SUP2_turbo_7.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/MX_monday_ELC_normal_7_2025-09-28.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/CO_monday_ELC_normal_7_2025-09-28.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/PE_monday_ELC_normal_7_2025-09-28.csv
📁 Guardado en Drive: /content/drive/MyDrive/Events creation/CR_monday_ELC_normal_7_2025-09-28.csv
📁 Guardado e

In [ ]:
nombres_filtrados = [name for name in dfs_por_pais.keys() if dia_seleccionado in name.lower()]

MAPEO DE EVENTOS PARA EVENTS

In [ ]:
spreadsheet_generator = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY/edit?gid=934983762#gid=934983762')
offering_ws = spreadsheet_generator.worksheet("offering")
generator_ws = spreadsheet_generator.worksheet("generator")
offering_data = offering_ws.get_all_records()
generator_ws.clear()

{'spreadsheetId': '1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY',
 'clearedRange': 'generator!A1:AM992'}

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import calendar

# === 1. CARGAR ARCHIVOS DESDE LA CARPETA DE DRIVE ===
carpeta_drive = "/content/drive/MyDrive/Events creation"
archivos_csv = [f for f in os.listdir(carpeta_drive) if f.endswith('.csv')]

# === 2. FILTRAR ARCHIVOS QUE CUMPLAN CON EL FORMATO SUP1 ===
# === 2. FILTRAR ARCHIVOS QUE CUMPLAN CON EL FORMATO SUP1_normal O SUP1_turbo ===
# === 2. PATRÓN ÚNICO PARA SUP1 Y ELC ===
patron_eventos = re.compile(
    r'^[A-Z]{2}_(monday|tuesday|wednesday|thursday|friday)_(SUP1)_(?:[1-9]|1[0-2])\.csv$'
)

# === 3. CARGAR ARCHIVOS QUE COINCIDAN CON SUP1 ===
dfs_filtrados = {}
for archivo in archivos_csv:
    if patron_eventos.match(archivo):
        ruta = os.path.join(carpeta_drive, archivo)
        nombre_df = archivo.replace('.csv', '')
        dfs_filtrados[nombre_df] = pd.read_csv(ruta)


# === 3. CARGAR DATA DE OFFERING ===
spreadsheet_generator = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY/edit?gid=934983762#gid=934983762')
offering_ws = spreadsheet_generator.worksheet("offering")
generator_ws = spreadsheet_generator.worksheet("generator")
offering_data = offering_ws.get_all_records()

# === 4. FECHAS Y MESES ===
fecha_actual = datetime.today()
fecha_reg_start = fecha_actual.strftime('%Y-%m-%d')
fecha_event_start = (fecha_actual + timedelta(days=2)).strftime('%Y-%m-%d')
fecha_reg_end = (fecha_actual + timedelta(days=14)).strftime('%Y-%m-%d')
fecha_event_end = (fecha_actual + timedelta(days=15)).strftime('%Y-%m-%d')

meses_es = {
    1: "ENERO", 2: "FEBRERO", 3: "MARZO", 4: "ABRIL", 5: "MAYO", 6: "JUNIO",
    7: "JULIO", 8: "AGOSTO", 9: "SEPTIEMBRE", 10: "OCTUBRE", 11: "NOVIEMBRE", 12: "DICIEMBRE"
}

columns = [
    "Month","Country","Event reviewer","B APP","BD APP","Gattaran Batch Registration Tool",
    "Event","Event theme","Internal act name","Aux","Registration Start Date","Registration End Date",
    "Event Start Date","Event End Date","Mon","Tues","Wed","Thurs","Fri","Sat","Sun",
    "Start Hour","End  Hour","DiscountType","Discount","RewardMode","Rburn % Discount Range (On Top Of)",
    "Reward Ratio","Max Budget per Store","ShopFilter (.csv)","AdImage (.png) (online)",
    "Ad description (Online)","Ad text (online)","Item type",
    "Restrict the phenomenon of “offering discounts after raising item prices”?",
    "Min Items","Max items","Min Price","Max Price"
]

# === 5. CONSTRUIR FILAS PARA LOS ARCHIVOS SUP1 ===
rows = []
for name in dfs_filtrados:
    parts = name.split('_')
    country, weekday, event, month = parts
    nombre_archivo_csv = f"{name}"  # 👈 Aquí obtienes el nombre del archivo

    if month.isdigit():
        month_num = int(month)
        month_en = calendar.month_name[month_num].upper()
        month_start = calendar.month_name[month_num]
        month_es = meses_es[month_num]
    else:
        month_en = month.upper()
        month_es = month.upper()

    internal_act_name = f"ACQ_{event}_{month_es}_DD_"
    clave_oferta = f"{country}_{event}_{month_en}"

    discount = reward_ratio = discountType = ""
    max_budget = min_items = max_items = min_price = max_price = ""
    ad_image = ad_description = ad_text = ""

    for fila in offering_data:
        if fila.get("file", "").strip().upper() == clave_oferta:
            discountType = fila.get("DiscountType", "")
            discount = fila.get("Discount", "")
            reward_ratio = fila.get("Reward Ratio", "")
            max_budget = fila.get("Max Budget per Store", "")
            min_items = fila.get("Min Items", "")
            max_items = fila.get("Max items", "")
            min_price = fila.get("Min Price", "")
            max_price = fila.get("Max Price", "")
            ad_image = fila.get("AdImage (.png) (online)", "")
            ad_description = fila.get("Ad description (Online)", "")
            ad_text = fila.get("Ad text (online)", "")
            break

    row = [
        month_start, country, "erickvargassaldana", False, False, True,
        event, "¡Bienvenido a DiDi Food!", internal_act_name, "",
        fecha_reg_start, fecha_reg_end, fecha_event_start, fecha_event_end,
        True, True, True, True, True, True, True,
        0, 24, discountType, discount, "Original Price Mode", "",
        reward_ratio, max_budget, nombre_archivo_csv,  # 👈 Aquí va el nombre del archivo
        ad_image, ad_description, ad_text,
        "All", "No", min_items, max_items, min_price, max_price
    ]
    rows.append(row)

# === 6. SUBIR A HOJA DESTINO ===
generator_ws.update(values=[columns], range_name='A1')
generator_ws.update(values=rows, range_name='A2')


{'spreadsheetId': '1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY',
 'updatedRange': 'generator!A2'}

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import calendar

# === 1. CARGAR ARCHIVOS DESDE LA CARPETA DE DRIVE ===
carpeta_drive = "/content/drive/MyDrive/Events creation"
archivos_csv = [f for f in os.listdir(carpeta_drive) if f.endswith('.csv')]

# === 2. FILTRAR ARCHIVOS QUE CUMPLAN CON EL FORMATO SUP2 ===
# === 2. FILTRAR ARCHIVOS QUE CUMPLAN CON EL FORMATO SUP2_normal O SUP2_turbo ===
# === 2. PATRÓN ÚNICO PARA SUP2 Y ELC ===
patron_eventos = re.compile(
    r'^[A-Z]{2}_(monday|tuesday|wednesday|thursday|friday)_(SUP2)_(normal|turbo)_(?:[1-9]|1[0-2])\.csv$'
)

# === 3. CARGAR ARCHIVOS QUE COINCIDAN CON SUP2 o ELC ===
dfs_filtrados = {}
for archivo in archivos_csv:
    if patron_eventos.match(archivo):
        ruta = os.path.join(carpeta_drive, archivo)
        nombre_df = archivo.replace('.csv', '')
        dfs_filtrados[nombre_df] = pd.read_csv(ruta)


# === 3. CARGAR DATA DE OFFERING ===
spreadsheet_generator = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY/edit?gid=934983762#gid=934983762')
offering_ws = spreadsheet_generator.worksheet("offering")
generator_ws = spreadsheet_generator.worksheet("generator")
offering_data = offering_ws.get_all_records()

# === 4. FECHAS Y MESES ===
fecha_actual = datetime.today()
fecha_reg_start = fecha_actual.strftime('%Y-%m-%d')
fecha_event_start = (fecha_actual + timedelta(days=2)).strftime('%Y-%m-%d')
fecha_reg_end = (fecha_actual + timedelta(days=14)).strftime('%Y-%m-%d')
fecha_event_end = (fecha_actual + timedelta(days=15)).strftime('%Y-%m-%d')

meses_es = {
    1: "ENERO", 2: "FEBRERO", 3: "MARZO", 4: "ABRIL", 5: "MAYO", 6: "JUNIO",
    7: "JULIO", 8: "AGOSTO", 9: "SEPTIEMBRE", 10: "OCTUBRE", 11: "NOVIEMBRE", 12: "DICIEMBRE"
}

columns = [
    "Month","Country","Event reviewer","B APP","BD APP","Gattaran Batch Registration Tool",
    "Event","Event theme","Internal act name","Aux","Registration Start Date","Registration End Date",
    "Event Start Date","Event End Date","Mon","Tues","Wed","Thurs","Fri","Sat","Sun",
    "Start Hour","End  Hour","DiscountType","Discount","RewardMode","Rburn % Discount Range (On Top Of)",
    "Reward Ratio","Max Budget per Store","ShopFilter (.csv)","AdImage (.png) (online)",
    "Ad description (Online)","Ad text (online)","Item type",
    "Restrict the phenomenon of “offering discounts after raising item prices”?",
    "Min Items","Max items","Min Price","Max Price"
]

# === 5. CONSTRUIR FILAS PARA LOS ARCHIVOS SUP2 ===
rows = []
for name in dfs_filtrados:
    parts = name.split('_')
    country, weekday, event, tipo, month = parts
    nombre_archivo_csv = f"{name}"  # 👈 Aquí obtienes el nombre del archivo

    if month.isdigit():
        month_num = int(month)
        month_en = calendar.month_name[month_num].upper()
        month_start = calendar.month_name[month_num]
        month_es = meses_es[month_num]
    else:
        month_en = month.upper()
        month_es = month.upper()

    tipo_upper = tipo.upper()
    internal_act_name = f"ACQ_{event}_{month_es}_{tipo_upper}_DD_"
    clave_oferta = f"{country}_{event}_{tipo_upper}_{month_en}"

    discount = reward_ratio = discountType = ""
    max_budget = min_items = max_items = min_price = max_price = ""
    ad_image = ad_description = ad_text = ""

    for fila in offering_data:
        if fila.get("file", "").strip().upper() == clave_oferta:
            discountType = fila.get("DiscountType", "")
            discount = fila.get("Discount", "")
            reward_ratio = fila.get("Reward Ratio", "")
            max_budget = fila.get("Max Budget per Store", "")
            min_items = fila.get("Min Items", "")
            max_items = fila.get("Max items", "")
            min_price = fila.get("Min Price", "")
            max_price = fila.get("Max Price", "")
            ad_image = fila.get("AdImage (.png) (online)", "")
            ad_description = fila.get("Ad description (Online)", "")
            ad_text = fila.get("Ad text (online)", "")
            break

    row = [
        month_start, country, "erickvargassaldana", True, True, True,
        event, "¡Bienvenido a DiDi Food!", internal_act_name, "",
        fecha_reg_start, fecha_reg_end, fecha_event_start, fecha_event_end,
        True, True, True, True, True, True, True,
        0, 24, discountType, discount, "Original Price Mode", "",
        reward_ratio, max_budget, nombre_archivo_csv,  # 👈 Aquí va el nombre del archivo
        ad_image, ad_description, ad_text,
        "All", "No", min_items, max_items, min_price, max_price
    ]
    rows.append(row)

# === 6. SUBIR A HOJA DESTINO ===
generator_ws.update(values=[columns], range_name='A1')
generator_ws.update(values=rows, range_name='A2')


{'spreadsheetId': '1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY',
 'updatedRange': 'generator!A2:AM7',
 'updatedRows': 6,
 'updatedColumns': 39,
 'updatedCells': 234}

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import calendar

# === 1. CARGAR ARCHIVOS DESDE LA CARPETA DE DRIVE ===
carpeta_drive = "/content/drive/MyDrive/Events creation"
archivos_csv = [f for f in os.listdir(carpeta_drive) if f.endswith('.csv')]

# === 2. PATRÓN PARA ELC_normal o ELC_turbo ===
patron_elc = re.compile(
    r'^[A-Z]{2}_(monday|tuesday|wednesday|thursday|friday)_(ELC)_(normal|turbo)_(?:[1-9]|1[0-2])_\d{4}-\d{2}-\d{2}\.csv$'
)


# === 3. CARGAR ARCHIVOS QUE COINCIDAN CON ELC ===
dfs_elc = {}
for archivo in archivos_csv:
    if patron_elc.match(archivo):
        ruta = os.path.join(carpeta_drive, archivo)
        nombre_df = archivo.replace('.csv', '')
        dfs_elc[nombre_df] = pd.read_csv(ruta)

# === 4. CARGAR DATA DE OFFERING ===
spreadsheet_generator = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY/edit?gid=934983762#gid=934983762')
offering_ws = spreadsheet_generator.worksheet("offering")
generator_ws = spreadsheet_generator.worksheet("generator")
offering_data = offering_ws.get_all_records()

# === 5. FECHAS Y MESES ===
fecha_actual = datetime.today()
fecha_reg_start = fecha_actual.strftime('%Y-%m-%d')
fecha_event_start = (fecha_actual + timedelta(days=2)).strftime('%Y-%m-%d')


meses_es = {
    1: "ENERO", 2: "FEBRERO", 3: "MARZO", 4: "ABRIL", 5: "MAYO", 6: "JUNIO",
    7: "JULIO", 8: "AGOSTO", 9: "SEPTIEMBRE", 10: "OCTUBRE", 11: "NOVIEMBRE", 12: "DICIEMBRE"
}

columns = [
    "Month","Country","Event reviewer","B APP","BD APP","Gattaran Batch Registration Tool",
    "Event","Event theme","Internal act name","Aux","Registration Start Date","Registration End Date",
    "Event Start Date","Event End Date","Mon","Tues","Wed","Thurs","Fri","Sat","Sun",
    "Start Hour","End  Hour","DiscountType","Discount","RewardMode","Rburn % Discount Range (On Top Of)",
    "Reward Ratio","Max Budget per Store","ShopFilter (.csv)","AdImage (.png) (online)",
    "Ad description (Online)","Ad text (online)","Item type",
    "Restrict the phenomenon of “offering discounts after raising item prices”?",
    "Min Items","Max items","Min Price","Max Price"
]

# === 6. CONSTRUIR FILAS PARA LOS ARCHIVOS ELC ===
rows = []
for name in dfs_elc:
    parts = name.split('_')
    country, weekday, event, tipo, month, end = parts
    nombre_archivo_csv = f"{name}"  # 👈 Aquí guardamos el nombre original

    if month.isdigit():
        month_num = int(month)
        month_en = calendar.month_name[month_num].upper()
        month_start = calendar.month_name[month_num]
        month_es = meses_es[month_num]
    else:
        month_en = month.upper()
        month_es = month.upper()

    tipo_upper = tipo.upper()
    clave_oferta = f"{country}_{event}_{tipo_upper}_{month_en}".upper()
    internal_act_name = f"ACQ_{event}_{month_es}_{tipo_upper}_DD_"
    end_dt = datetime.strptime(end, '%Y-%m-%d')  # Convertir string a datetime
    fecha_reg_end = (end_dt - timedelta(days=1)).strftime('%Y-%m-%d')
    end = end_dt.strftime('%Y-%m-%d')


    # Buscar en hoja offering
    discount = reward_ratio = discountType = ""
    max_budget = min_items = max_items = min_price = max_price = ""
    ad_image = ad_description = ad_text = ""
    found = False

    for fila in offering_data:
        if fila.get("file", "").strip().upper() == clave_oferta:
            discountType = fila.get("DiscountType", "")
            discount = fila.get("Discount", "")
            reward_ratio = fila.get("Reward Ratio", "")
            max_budget = fila.get("Max Budget per Store", "")
            min_items = fila.get("Min Items", "")
            max_items = fila.get("Max items", "")
            min_price = fila.get("Min Price", "")
            max_price = fila.get("Max Price", "")
            ad_image = fila.get("AdImage (.png) (online)", "")
            ad_description = fila.get("Ad description (Online)", "")
            ad_text = fila.get("Ad text (online)", "")
            found = True
            break

    if not found:
        print(f"⚠️ No se encontró '{clave_oferta}' en la hoja 'offering'.")

    row = [
        month_start, country, "erickvargassaldana", True, True, True,
        event, "Disfruta promos con DiDi Food", internal_act_name, "",
        fecha_reg_start, fecha_reg_end, fecha_event_start, end,
        True, True, True, True, True, True, True,
        0, 24, discountType, discount, "Original Price Mode", "",
        reward_ratio, max_budget, nombre_archivo_csv, ad_image, ad_description,
        ad_text, "All", "No", min_items, max_items, min_price, max_price
    ]
    rows.append(row)

# === 7. SUBIR A HOJA DESTINO (agregando debajo de lo ya existente) ===
existing_values = generator_ws.get_all_values()
start_row = len(existing_values) + 1

# Subir filas a partir de la fila vacía sin especificar columna final
range_name = f"A{start_row}"

# ✅ Esto asegura que todas las columnas se escriban correctamente
generator_ws.update(values=rows, range_name=range_name)

print(f"\n✅ Archivos ELC procesados: {len(dfs_elc)}")
print(f"✅ Filas añadidas a partir de la fila {start_row}: {len(rows)}")




✅ Archivos ELC procesados: 7
✅ Filas añadidas a partir de la fila 8: 7


In [ ]:
#Formato al mapeo de eventos previo
def num_a_letras_excel(n):
    """Convierte un índice numérico (1-based) a letras estilo Excel."""
    letras = ""
    while n:
        n, r = divmod(n - 1, 26)
        letras = chr(65 + r) + letras
    return letras


In [ ]:
# === 8. BORRAR CONTENIDO DE COLUMNAS 'Aux' Y 'Rburn % Discount Range (On Top Of)' ===

# Obtener encabezados
headers = existing_values[0]

# Buscar índices (1-based para Google Sheets)
aux_index = headers.index("Aux") + 1
rburn_index = headers.index("Rburn % Discount Range (On Top Of)") + 1

# Convertir a letras estilo Excel
aux_col_letter = num_a_letras_excel(aux_index)
rburn_col_letter = num_a_letras_excel(rburn_index)

# Determinar el rango de filas a limpiar (desde la fila 2)
last_row = len(existing_values)

# Rango en notación A1 para ambas columnas
aux_range = f"{aux_col_letter}2:{aux_col_letter}{last_row}"
rburn_range = f"{rburn_col_letter}2:{rburn_col_letter}{last_row}"

# Limpiar contenido de ambas columnas
generator_ws.batch_clear([aux_range, rburn_range])


{'spreadsheetId': '1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY',
 'clearedRanges': ['generator!J2:J7', 'generator!AA2:AA7']}

In [ ]:
# === 9. CENTRAR TODAS LAS COLUMNAS EXCEPTO ALGUNAS ===

# Columnas a excluir
excluir_columnas = ["Event theme","Internal act name", "Min Items", "Max items", "Min Price", "Max Price"]

# Función para convertir índice a letra estilo Excel
def num_a_letras_excel(n):
    letras = ""
    while n:
        n, r = divmod(n - 1, 26)
        letras = chr(65 + r) + letras
    return letras

# Recalcular total de filas actuales (incluyendo las que acabamos de insertar)
total_filas = len(generator_ws.get_all_values())

# Obtener encabezados
headers = generator_ws.row_values(1)
requests = []

for idx, header in enumerate(headers):
    if header not in excluir_columnas:
        requests.append({
            "repeatCell": {
                "range": {
                    "sheetId": generator_ws._properties['sheetId'],
                    "startRowIndex": 1,  # Desde fila 2 (0-indexed)
                    "endRowIndex": total_filas,
                    "startColumnIndex": idx,
                    "endColumnIndex": idx + 1
                },
                "cell": {
                    "userEnteredFormat": {
                        "horizontalAlignment": "CENTER"
                    }
                },
                "fields": "userEnteredFormat.horizontalAlignment"
            }
        })

# Aplicar centrado
generator_ws.spreadsheet.batch_update({"requests": requests})


{'spreadsheetId': '1nKuDiWa7PNNAt7EH8Ar-U9xZDiGn3gWGLyDWg0Vk2NY',
 'replies': [{},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {}]}